In [1]:

import os #여기부터 실행하세요. one drive에서 파일 다운받아두세요. 
import re
import openpyxl
import calendar
from bs4 import BeautifulSoup
from pathos.multiprocessing import ProcessingPool as Pool
import pandas as pd
import time


cal = calendar.Calendar()

query = "여성"
query2 = "#2_" + query
year = 2019
xlist = list()

for month in range(1, 13):
        monthdays = [d for d in cal.itermonthdays(year, month) if d != 0]
        for day in monthdays:
            dd = str(year) + str(month).zfill(2) + str(day).zfill(2)
            xlsx_name = os.path.join(os.path.dirname(os.getcwd()), 'xlsx_data', '#2_html2text', query2, '#2_'+ str(year) + '_'+query, 
                                     '{}.xlsx'.format("#2_"+dd+"_"+query))
            xlist.append(xlsx_name)


In [167]:
def replace_sub(text):
                
        from nltk.tokenize import regexp_tokenize
        from string import punctuation

        tlist, ttlist= list(), list()
        text = regexp_tokenize(text, r'[^\s\.][^\.\n]+')
        for _ in text:
            if len(_) > 11:
                tlist.append(_)
                
        text = ".\n".join(tlist) + "."

        text = re.sub(r'\d+\.(\n)\d+', r'', text)
        text = re.sub(r'\b(\w|[.])+@(?:[.]?\w+)+\b', ' ', text)
        text = re.sub(r'\bhttps?://\w+(?:[.]?\w+)+\b', ' ', text)
        text = re.sub(r'[^A-Za-z0-9가-힣ㄱ-하-ㅣ\.?!\n ]', ' ', text)
        text = re.sub(r'\b[a-z][A-Za-z0-9]+\b', ' ', text)
        text = re.sub(r'\s{2,}', ' ', text) 
        text = re.sub(r'[\(\[].*?[\)\]]', '', text)
        text = re.sub('【.*】','',text)
        text = re.sub(r'"\\#\\$%\\&\'\\(\\)\\*\\+,\\-\/:;<=>\@\\[\\\\\\]\\^_`\\{\\|\\}\\~', '', text)
        # 여기부터 새로운 태그 추가
        #신문사 이름은 나중에 한꺼번에 뺄 것이니 추가하지 말 것
        text = re.sub(r"-\d+-", '', text) # 이거 전에 '-'제거하면 안됨.
        text = re.sub(r"\b\w{3}\s기자", '', text)
        text = re.sub(r'\s?[a-z]\s', ' ', text)
        text = re.sub(r'\s+\d+px?', '', text)
        text = re.sub(r'\s\d+\s\d+\s\d+\s', '', text)
        text = re.sub(r'\s?(\d+\s)', '', text)
        text = re.sub(r'((\d+[a-z])+)', '', text)
        text = re.sub(r'[A-Z0-9a-z][A-Z0-9a-z]', '', text)
        text = re.sub(r'\b[A-Z0-9]\b', '', text)
        
        text = re.sub('해당 언론사가.*?가시겠습니까?','',text,flags=re.DOTALL)
        text = re.sub('무단전재.*?분류했습니다.','',text,flags=re.DOTALL)
        text = re.sub('기사 섹션 분류 안내.*?따르고 있습니다.','',text,flags=re.DOTALL)
        text = re.sub('네이버에서.*?클릭','',text,flags=re.DOTALL)
        text = re.sub('제한된 글자수 자를.*?비밀번호 가 일치 하지 않습니다.','',text,flags=re.DOTALL)
        text = re.sub('네이버에서.*?채널 구독하기','',text,flags=re.DOTALL)
        
        text = text.replace('출처', '').replace('방송캡처', '').replace('-', '').replace('오류를 우회하기 위한 함수 추가.', '')
        text = text.replace('동영상 뉴스.', '').replace('아래 텍스트는 실제 방송 내용과 차이가 있을 수 있으니 보다 정확한 내용은 방송으로 확인하시기 바랍니다.', '')
        text = text.replace(' 사진 ', ' ').replace('경남신문의 콘텐츠는 저작권법의 보호를 받는 바 무단전재 복사 재배포를 금합니다.', '')
        text = text.replace('돋움', '').replace('메인이미지', '').replace('. p .', "\n").replace('정상처리되지 않았습니다.', '')
        text = text.replace(' p ', '\n').replace('IMGFLOATING ', '').replace('캡쳐 ', '').replace(' P ', '').replace('INSERT RANDOM NUMBER HERE', '')
        text = text.replace('OptimaA Scrip Ta', '').replace('시원하게 털어드립니다.', '').replace('저작권자 인사이트 무단전재 및 재배포 금지 .', '')
        text = text.replace('수직배너 와이드', '').replace('www.', '').replace('동영상이 안 나오면', '').replace('여기를 누르시면 동영상을 별도의 새 창에서 볼 수 있습니다.', '')
        
        text = text.replace('취재기자 심충만 방송일조회', '').replace('게시글을 r로 보내기', '').replace('게시글을 으로 보내기', '').replace('게시글을 로 보내기', '')
        text = text.replace('이 글을 페이스북으로 퍼가기', '').replace('이 글을 트위터로 퍼가기', '').replace('이 글을 카카오스토리로 퍼가기', '').replace('이 글을 밴드로 퍼가기', '')
        text = text.replace('요즘 게시글을 요즘으로 보내기', '').replace('E에 있는 정보를 기본으로 가져오며', '').replace('해당 테이블에 등록된 정보가 없을 시', '')
        text = text.replace('모바일에 등록된 4를 가져온다', '').replace('롱리브더킹 임시 해결', '').replace('오른쪽 마우스 부정클릭 방지', '')
        text = text.replace('카툰 유료화 안내 표시', '').replace('기사본문 광고 제거', '').replace('! !','').replace('? ?', '')
        text = text.replace('기자의 다른 기사 보기','').replace('뉴광고 기사 배너지면 가변','').replace('기사 모바일 우측상단','')
        text = text.replace('게시물을 삭제하시겠습니까?','').replace('로그인하신 후에 이용해 주시기 바랍니다!','').replace('본문상단우측','').replace('공식 이미지','')
        text = text.replace('기자가 쓴 다른기사 보기','').replace('해당 언론사가 주요기사로 직접 선정한 기사입니다','').replace('언론사 편집판 바로가기','')
        text = text.replace('게티이미지뱅크','').replace('함수 를 구현합니다','').replace('언론사는 개별 기사를 2개 이상 섹션으로 중복 분류할 수 있습니다','')
        text = text.replace('헉! 소리나는 스!토리 뉴스 헉스','').replace('클릭해 뉴스 들어간다 뉴스쿨','').replace('이 기사를 추천합니다','')
        text = text.replace('본문 텍스트A','').replace('포토 위젯 시작 문의','').replace('포토 위젯 종료 문의','').replace('게티이미지','')
        text = text.replace('이전데이터에서 사진이 추가될 경우 본문 삽입처리','').replace('기사본문 오른쪽 정렬','')
        text = text.replace('본문내 상단우측','').replace('아티클 공통 기자','').replace('아티클 공통 공유 및 댓글','')
        text = text.replace('새창 페이스북으로 이동 페이스북','').replace('새창 트위터로 이동 트위터','').replace('새창 카카오스토리로 이동 카카오스토리','')
        text = text.replace('댓글로 이동 댓글 수','').replace('아티클 공통 본문','').replace('아티클 공통 부제','')
        text = text.replace('기사 페이지 본문내우측','').replace('이미지 확대보기','').replace('에스엔에스 공유경제신문','').replace('에스엔에스에스비에스화면','') 
        
        text = text.replace('저작권자 입법국정전문지 더리더 e 무단전재 및 재배포 금지 .','').replace('머니투데이 더리더','')
        text = text.replace('사진','').replace('에너지경제 온에어','').replace('뉴스','')
        text = text.replace('제공 안양시 안양 파이낸셜','').replace('매직스트로베리사운드 제공.','')
        text = text.replace('정하은기자','').replace('캡처 헤럴드P','').replace('경기도청 전경 군포시민신문.','').replace('독자가 내는 소중한 월 1천원 구독료는 군포시민신문 대부분의 재원이자 올바른 지역언론을 지킬 수 있는 힘입니다.','')
        text = text.replace('충북 제천경철서 홈페이지 캡처.','').replace('페이스북 캡처','').replace('김민성 인턴기자','').replace('S썸바이벌캡처','')
        text = text.replace('저작권자 스타트업투데이 무단전재 및 재배포 금지.','').replace('로그인 후 이용 가능합니다.','').replace('인스타그램 캡처','')
        text = text.replace('이 에디터의 다른 글 보기.','').replace('저작권자 아트인사이트 .','').replace('추천 비추천은 회원만 가능합니다.','').replace('값이 제대로 넘어오지 않았습니다.','').replace('등록된 코멘트가 없거나 코멘트 글이 아닙니다.','')
        text = text.replace('자신의 글에는 추천 또는 비추천 하실 수 없습니다.\n이미 추천 하신 글 입니다.\n이미 비추천 하신 글 입니다.','').replace(' 페이스북 페이지 캡처','')
        text = text.replace('디지털 콘텐츠 이용규칙 보기','').replace('조혜진기자 앵커','').replace('권은주기자','').replace('! 뉴데일리 뉴데일리','').replace('출품 여성작가들과 임종국 교수 오른쪽','')
        text = text.replace('출품 작가 김미숙 김종태 신보경 신소윤 안선문 인상희 임순옥 임윤상 한경진 홍은희 홍태돈 지도교수 임종국 .','')
        text = text.replace('전시회는 7월8일까지 서울교대 샘미술관 아카데미 실용반 활동영상 .','').replace('E평생교육원실용반그룹전 g영상','').replace('A아래에 작가별 대표작들을 소개한다','')
        text = text.replace('무단전재 재배포 금지','').replace('보도자료 및 기사제보','').replace('자유민주 시장경제의 파수꾼 뉴데일리 ','').replace('기사','')
        text = text.replace('인터플레이 중앙중단','')
        text = text.replace('라이브리 댓글 작성을 위해 를 활성화 해주세요','').replace('뉴데일리 댓글 운영정책','')
        text = text.replace('왼쪽부터 임호천씨박주식씨김해정씨.','').replace('이상훈 우철훈 김정근 선임기자','').replace('제공 e','')
        text = text.replace('우창범 트위터 웹데일리','').replace('오애희 웹데일리 기자','')
        text = text.replace('저작권자 웹데일리 무단 전재 및 재배포 금지 .','').replace('저작권자 기호일보 무단전재 및 재배포 금지','').replace('정상처리되지 않았습니다!','')
        text = text.replace('원하는곳에 붙여넣기','').replace('하루 김명민 이종석과 다정하게 한 컷','').replace(' 모바일 우측상단','').replace('김유진 공유경제신문 기자','').replace('공유경제가 제공하는 콘텐츠에 대해 독자는 친근하게 접근할 권리와 정정ㆍ반론ㆍ추후 보도를 청구 할 권리가 있습니다','').replace('저작권자 공유경제신문 무단 전재 및 재배포 금지','')
        text = text.replace('이건희 공유경제신문 기자','').replace('기부와 문화인류학 기부의 성립조건','').replace('공유 시선 공동체 자치 가 공유지의 비극 해결','')
        text = text.replace('공유 시선 서구 문학에서 장애는 어떻게 재현되나.\n공경기획 기업의 공유가치창출 기업가치를 높이는 경영전략.','').replace('산책 선순환 방식의 기부문화 나와야','')
        text = text.replace('기부와 문화인류학 기부와 경제.','').replace('공경기획 공유경제 플랫폼의 다양화로 협력적 소비 이끌어','').replace('산책 프로보노 사회의 변화를 추구하는 기부 방식','')
        text = text.replace('공경기획 공유경제 서비스 영역의 확대','').replace('공경기획 숙박 공유 플랫폼 지역사회 경제 기여','').replace('공경기획 공유경제는 소유에서 시작된다','')
        text = text.replace('저작권자 대구 경북 대표지역언론 대구일보 ','').replace('폴리 박현 기자','').replace('월 일모바일한경 에 게재된 입니다','').replace('모바일한경 더보기 ','').replace('모바일한경은 프리미엄 채널입니다','').replace('한국정책방송원 무단전재 및 재배포 금지','').replace('대한 9회 클립영상','').replace('영상취재 김명신 홍성주 영상편집 양세형','')
        text = text.replace('저작권자 로이슈 무단 전재 및 재배포 금지','').replace('제보 및 보도자료 제공','').replace('에디터에서 들어간 가 아닐 경우 태그 씌우지 않음','').replace('이 는 언론사에서 사회 섹션으로 분류했습니다','').replace('네이버에서 뉴시스 채널 구독하기 뉴시스 빅데이터 주가시세표 바로가기 저작권자 공감언론 뉴시스통신사','').replace('의 섹션 정보는 해당 언론사의 분류를 따르고 있습니다','').replace('이 는 사용자 추천 으로 모바일 메인 판에 노출된 이력이 있습니다','')
        text = text.replace('무단전재 및 재배포 금지','').replace('저작권자 헤드라인제주','')
        text = text.replace('메인페이지가 로드됩니다','').replace('본문 중하단 광고 서인선','').replace('의견글을 삭제하시겠습니까?','').replace('제공 의정부시 의정부 스포츠서울','').replace('후원 금액을 선택해주세요','').replace('정기후원 페이지로 이동합니다','')
        text = text.replace('팝업 차단를 해제하고 이용하시기 바랍니다','').replace('팝업창을 활성화 해주세요','').replace('가 마음에 드셨나요? ','').replace('학교비정규직노조 울산지부 제공','').replace('문체부 게시물 분석와 무관한 이미지입니다','').replace('이미지 로이터연합','').replace('류원혜 인턴기자','').replace('김고금평 기자','').replace('타다 문제 계약해지 성 인지 교육 강화 저작권자','').replace('무단복제재배포 금지 네이버 홈에서 C 채널 구독하기 엠빅분토론 심상정양경규 진보에 길을 묻다 살충제 이겨낸 바퀴벌레의 초특급 진화','').replace('이 는 모바일 메인 판 에 노출된 이력이 있습니다','')
        text = text.replace('제한된 글자수 자를 초과하였습니다','').replace('닉네임 을 입력해 주세요','').replace('비밀번호 를 입력해 주세요','').replace('건전한 댓글문화 정착을 위해 이용에 주의를 부탁드립니다','').replace('명예훼손 개인정보 유출 인격권 침해 허위사실 유포 등은 이용약관 및 관련법률에 의해 제재를 받을 수 있습니다.','').replace('댓글내용 을 입력해 주세요','').replace('양수미 공유경제신문 기자','').replace('양수미 공유경제신문 기자','').replace('무단 전재 및 재배포 금지','').replace('꿈을 담는 캔버스 채널A ','').replace('저작권 언론사 하이닥 무단 전재 및 재배포 금지','').replace('저작권자 패션엔미디어','').replace('저작권자 중앙일보플러스 무단전재 및 재배포금지 ','').replace('채널A 무료 보기 네이버에서 채널A 구독하기 보기 판결의 재구성 모자 실종 반전의 잔혹극 신고자가 살해 용의자','')
        text = text.replace('김수지기자','').replace('무단 전재재배포 금지','').replace('공보닷컴 트위터 캡처 헤럴드경제 정지은 인턴기자','').replace('이미지 연합 아시아경제 조슬기나 기자','')
        text = text.replace('제공 삼성화재 스포츠서울 삼성화재는 어','').replace(' 열매 아프리카 방송 캡처온라인','').replace('저작권 한국일보','').replace('이지웅 웹데일리 기자','')
        text = text.replace('글 이혁재 소아시한의원 원장 주간경향','').replace('글 이혁재 소아시한의원 원장 주간경향','').replace('의 디지털이용규칙에 따른 저작권을 행사합니다','')
        text = text.replace('제공 아메바컬쳐아메바컬쳐','').replace('위 의 법적인 책임과 권한은 에듀동아에 있습니다','').replace('에듀동아 전수완 인턴 기자','').replace('인라이플','').replace('저작권자 무단복제재배포 금지','').replace('등록된 댓글이 없습니다','')
        text = text.replace('로그인 후 댓글 등록이 가능합니다','').replace('지금 로그인 하시겠습니까','').replace('주소를 클립보드에 복사하시겠습니까?','').replace('이 글의 L주소입니다.','').replace('를 눌러 클립보드로 복사하세요','').replace(' 인터플레이 ','').replace('중앙중단','')
        text = text.replace('보도자료제목','').replace('제공 아메바컬쳐힙합그룹','').replace('스포츠서울 김자영기자','').replace('로이슈가 제공하는 콘텐츠에 대해 독자는 친근하게 접근할 권리와 정정ㆍ반론ㆍ추후 보도를 청구 할 권리가 있습니다.','')     
        
        text = text.replace('C뉴스 제희원입니다.','').replace('취재기자 제희원 방송일조회.','').replace('뉴스돋보기 여성장애인으로 산다는 것 .','')
        text = text.replace('연합뉴스 김동욱입니다.','').replace('방송화면 캡처','').replace('방송 영상 캡처','').replace('윤소윤 인턴기자','').replace('최진실기자','')
        text = text.replace('지원하지 않는 브라우저 입니다.','').replace('백호튜터링','').replace('트위터 제공','').replace('?제공','')
        text = text.replace('트위터 보내기 트위터.','').replace('페이스북 보내기 페이스북.','').replace('네이트온 보내기 네이트온 보내기.','')
        text = text.replace('제작 이태호 연합','').replace('영상취재 김태형 영상편집 양세형 .','')
        text = text.replace('인사이트','').replace('배너지면','').replace('라이브리 프리미엄 설치코드','').replace('선거전 기존 코드','')
        text = text.replace('면 경제섹션','').replace('?p 뉴시스 웹데일리 .','').replace('신세계푸드 광주기아챔피언스필드 신세계푸드 제공','').replace('류빈 기자','')
        
        text = text.replace('스크랩하기','').replace('배너클릭','').replace('상단 배너 광고','').replace('내용 모바일 배너','').replace('하단 배너 광고','')
        text = text.replace('다른 사람의 권리를 침해하거나 명예를 훼손하는 댓글은 관련 법률에 의해 제제를 받을 수 있습니다.','')
        text = text.replace('타인에게 불쾌감을 주는 욕설 등 비하하는 단어들은 표시가 제한됨을 알려드립니다.','')
        text = text.replace('유튜브 계정 캡처','').replace('로그인 후 작성이 가능합니다','').replace('C광주방송 C를 검색해서 보실 수 있습니다.','')
        text = text.replace('방송 캡처','').replace('넷플릭스 제공','').replace('페이스북 .','').replace('트위터 .','').replace('구글플러스 .','')
        text = text.replace('?루갈 박선호 입대 전 막지막 화보 복근 드러낸 멋진 슈트룩','').replace('?김보라 블랙 티셔츠에 직접 T까지! 우주인 조안 화제 만발 .','').replace('?뉴발란스 김연아 허훈 김하성 오승환 참여 질주본능 캠페인 .','').replace('?지컷 빨라진 신상품 출시 예년 보다 빠른 간절기 컬렉션 전개 .','')
        text = text.replace('?제니퍼 알렉스 화제의 연상연하! 한여름 밤의 쿨한 데이트 커플룩 .','').replace('?엑셀시오르 어드벤처러의 정신과 영감 담은 하이커 출시 .','').replace('?안젤리나 졸리 막내 아들과 쇼핑 나들이! 하이라이트는 X마스크 .','').replace('?한예슬 이집트 여왕! 압도적인 존재감의 팜므파탈 뷰티룩 .','')
        text = text.replace('?레깅스의 반란! 멋스럽게 연출하는 일상 레깅스룩 .','').replace('?패션엔 포토 트롯 여신 송가인 기분좋은 퇴근길! 시선 끄는 퍼프 슬리브! .','').replace('?서예지 송윤아 둘다 우아해! 같은 후프 귀걸이 어디꺼? .','').replace('?멜빵 바지의 귀환! 다시 뜨는 추억의 오버롤 패션 .','')
        text = text.replace('?패션 업계 벌써 겨울! 다운 패딩 선판매에 나선 이유? .','').replace('?세계 2위 섬유의류 수출국 베트남의 위기! 세계 패션시장 위협하나? .','').replace('?푸마 강다니엘와 협업 스니커즈 컬렉션 전개 .','').replace('?모두 내가 결정해 스타일리스트 없이 옷 잘입는 스타 셀럽들 .','')
        text = text.replace('?케이트 미들턴 이유있는 패션 규칙! 컬러 드레스 향연.','').replace('?이효리 공효진 현아 벙거지 모자 벗킷햇 으로 끝내는 핫섬머룩 .','').replace('?메건 마클 왕실 떠나도 완판녀! 만원짜리 린넨 원피스 주문 폭주 .','').replace('?미스터트롯 임영웅 아웃도어 브랜드 밀레 모델 발탁 .','')
        text = text.replace('?티셔츠 하나면 끝! 지프 레트로 티셔츠 컬렉션 인기폭발 .','').replace('?패션엔 포토 가수 제시 눈누난나! 출근길의 도발적인 레깅스 핏! .','').replace('?지지 하디드 임신 7개월째.','').replace('아름다운 D라인 깜짝 공개 .','')
        text = text.replace('?케이트 미들턴 애정 뚝뚝 자녀 사랑! 3남매를 위한 엄마표 생축샷 .','').replace('?카일리 제너 딸사랑 과했나? 2살짜리 딸 명품 가방 구설수 .','').replace('?셀레나 고메즈 못말리는 빨간 맛 사랑! 레드룩 .','').replace('?그 옷 어디꺼 선미 깜짝 등장! 화사한 플라워 원피스 어디꺼? .','')
        text = text.replace('?밀레 임영웅 효과 볼까? 임영웅 빅로고 재킷 예약 판매 .','').replace('?메건 마클 여름에 더 끌려! 매력적인 스트라이프 스타일.','').replace('?이효리 군살 제로! 요가로 다져진 애슬레저 핏 .','').replace('아이디룩할머니 카디건이 유행? 대신 짧게 입고 하나만 입어야 인싸! .','')
        text = text.replace('쉬즈미스한혜진 빨라진 여름 패션고민 끝! 도시 여성의 우아한 섬머룩 .','').replace('?푸마 강다니엘과 협업 스니커즈 컬렉션 전개 .','')
        text = text.replace('아티클 공통 필진의 최신 글 보기','').replace('아티클 공통 주요','').replace('아티클 공통 추천','').replace('아티클 공통 공유','').replace('아티클 공통 태그','').replace('새창 구글플러스로 이동 구글','').replace('새창 핀터레스트로 이동 핀터레스트','').replace('새창 구글플러스로 이동 구글','').replace('새창 카카오톡으로 이동 카카오톡','')
        text = text.replace('새창 네이버블로그로 이동 네이버블로그','').replace('게시판 관리기준에 의거하여 작성해 주시기 바랍니다.','').replace('네이버 메인에서 구독하기','')
        text = text.replace('스크랩은 회원에게만 지원되는 기능입니다 .','').replace('이미 스크랩한 입니다.','').replace('로그인 후 이용 하실 수 있습니다.','').replace('홈페이지 캡처','')
        text = text.replace('메일보내기 기능은 회원에게만 제공되는 기능 입니다.','').replace('오늘을 진단하고 내일의 청을 제시하는 C 서브타이틀','').replace('앵커멘트','').replace('데스크 앵커','')
        text = text.replace('네이버에서 채널 구독하기','').replace('유튜브에서 N스타 채널 구독하고 선물 받아 가세요','').replace('모두에게 보여주고 싶은 라면','').replace('ㄴㄴㄴㄴㄴㄴㄴㄴㄴㄴㄴㄴㄴ','')
        text = text.replace(' 은 내용과 직접적인 관련이 없음.','').replace('이미지 제공 삼성물산패션 빅데이터','').replace('누르면 을 더 크게 볼 수 있습니다','')
        text = text.replace('에프엔엔터테인먼트 제공','').replace('o e','').replace('a c','').replace('편집자 주 본고는 자료 제공사에서 제공한 것으로 연합는 내용에 대해 어떠한 편집도 하지 않았음을 밝혀 드립니다.','')
        text = text.replace('페이스북 페이스북 공유하기','').replace('트위터 트위터 공유하기','').replace('라인 라인 공유하기','').replace('네이버블로그 네이버블로그 공유하기','').replace('네이버밴드 네이버밴드 공유하기','')
        text = text.replace('네이버 네이버 공유하기','').replace('페이스북 화면 캡처','')
        text = text.replace('? 박용수 다른 보기 .','').replace('?6년 만에 룸 하차하는 손석희 후임은 .','').replace('?장병완 홍준표 인지능력 공감 부족.','').replace('?기름값 올라 3주 만의 상승 .','')
        text = text.replace('?신한은행 채용비리 오늘 밤 구속 결정 .','').replace('재정지원으로 해결될까? .','').replace('?단독 해썹 가공장 곰팡이와 오염물질 매년 적발 .','').replace('?단독 식약처 해썹업체관리 부실 축산물 위생 비상 .','')
        text = text.replace('?단독 식약처 무허가 가공장 단속 적발돼도 해썹은 유지 .','').replace('?한국식품안전관리인증원 해썹관리 매년 허술하게 점검해 .','').replace('?축산물 가공장 금속검출기 고장 쇳조각 걸러내지 못해 .','').replace('?김부겸 거침없이 가겠다 .','')
        text = text.replace('?소비자연대 쿠팡에 방역비용 청구 검토해야 주장 .','').replace('?김도읍 추미애 장관 윤석열 검찰총장 권한 무력화 해 .','').replace('?홍준표 좌파들 뻔뻔 년전 대출받아 산 집은 한채다 .','').replace('?종합 이인영 어떤 경우에도 남북 북미간 대화 끊이지 않아야 .','')
        text = text.replace('주호영 여당 일하는 국회법 은 야당 무력화법 독재 고속도로 .','').replace('?단독 쿠팡의 거짓말 거짓을 진실로 포장 .','').replace('?윤건영 박지원 국정원장 내정 탕평인사의 끝판 .','')
        text = text.replace('?해썹 가공 작업장 곳곳에 곰팡이.','').replace('구멍 숭숭 해썹 심사 .','').replace('?단독 해썹 가공장 곰팡이와 오염물질 매년 적발 .','').replace('? 이메일무단수집거부 .','').replace('한국탐사보도저널리즘 .','')
        text = text.replace('경기도 성남시 분당구 서현로 0번길 경기 아.','').replace('발행인 편집인 박동경 .','').replace('청소년보호책임자 김 준 .','').replace('개인정보관리 책임자 이현기 .','').replace('개인정보관리 책임자 이현기 .','')
        text = text.replace('후플러스의 모든 콘텐츠 영상 는 저작권법의 보호를 받은바 무단 전재와 복사 배포 등을 금합니다.','').replace('? 류지원 다른 보기 .','').replace('?김태년 한명숙 사법농단 피해자 법무부가 진실 밝혀야 .','').replace('?육아휴직 8명 중 1명은 아빠 대기업 쏠림 은 여전 .','')
        text = text.replace('?의협복지부 충돌 예고 .','').replace('?음악 스트리밍 창작자 몫 인상 추진 음원 이용료 오를까 .','').replace('?서울 북부지법 화장실 내 민원인 개인정보 노출.','')
        text = text.replace('픽스 버즈 F블루투스 이어폰 .','').replace('픽스 밸런스 인바디 체중계 .','').replace('픽스 쿨 휴대용 선풍기 .','').replace('픽스 바디체크 인바디 체중계 .','')
        text = text.replace('좋아요 를 누르시면 제일 빠른 소식을 전달해 드립니다.','').replace('이 광고문제로 로딩이 안되는 오류 발생','').replace('상단의 배너를 누르시면 바이오센추리 r 원문을 보실 수 있습니다.','').replace('방송 화면 캡처','')
        text = text.replace('날개광고 여백광고 .','').replace('하단 메인 반복 모듈 .','')
        text = text.replace('모든 브라우저 신규 플레시플레이어 없음 .','').replace('구 익스버전 기존 플레시플레이어 있는듯.','').replace('당신의 제보가 가 됩니다 N은 여러분의 소중한 제보를 기다립니다.','')
        text = text.replace('페이스북 공유하기','').replace('구글플러스 구글플러스 공유하기','').replace('싸이월드 싸이월드 공유하기','').replace('마이피플 공유하기','').replace('네이트온쪽지 네이트온쪽지 공유하기','').replace('구글북마크 구글북마크 공유하기','')
        text = text.replace('다른 곳에 퍼가실 때는 아래 고유 링크 주소를 로 사용해주세요.','').replace('클립보드에 복사하였습니다.','').replace('n원하시는 곳에 v키로 붙여 넣으세요.','')
        text = text.replace('기자프로필 노출여부 설정','').replace('프로필은 하단에 위의 과 함께 제공됩니다.','').replace('기자프로필 노출여부 설정 e','')
        text = text.replace('아이프레임 레이어팝업을 위한 설정 k는','').replace('p의 바로 아래 저장','').replace('레이어팝업시 배경마스킹을 위한 레이어','')
        text = text.replace('아이프레임을 부모창 정 가운데 레이어팝업으로 띄우는 경우 자식창에 ','').replace('p를 바로 인클루드 해도 무방','').replace('화면의 높이와 너비를 구한다.','')
        text = text.replace('마스크의 높이와 너비를 화면 것으로 만들어 전체 화면을 채운다.','').replace('애니메이션 효과 일단 1초동안 까맣게 됐다가불투명도로 간다.','').replace('로그인 후 의견을 등록하시면 자신의 의견을 관리하실 수 있습니다.','')
        text = text.replace('내용 저작권 등 다른 사람의 권리를 침해하거나 명예를 훼손하는 게시물은 이용약관 및 관련법률에 의해 제재를 받으실 수 있습니다.','')
        text = text.replace('나눔바른고딕 i e','')
        text = text.replace('서울경제를 만나보세요저작권자 서울경제 ','').replace('본문 내용이 없는 입니다','').replace('서울경제 바로가기 내가 살 아파트 실거래가 봇으로 네이버로 서울경제썸 구독하기 네이버 메인에서 ','').replace('저작권자 데일리시큐','')
        text = text.replace('프레시안 정기후원 네이버 프레시안 채널 구독 프레시안 제보t','').replace('저작권 언론사 하이닥','').replace('이 는 정보공유라이선스 영리금지 를 따릅니다','').replace('한국경제 이지효입니다','').replace('세상에 이런일이 파스 저작권자 파이낸셜','').replace('집계 기간 동안 추천을 많이 받은 는 네이버 자동 배열 영역에 추천 요소로 활용됩니다','')
        text = text.replace('제갈수만 기자','').replace('뉴시스 빅데이터 주가시세표 바로가기 저작권자 공감언론 뉴시스통신사','').replace('서울 바로가기 인기 무료만화 네이버에서 서울신문 구독하기 서울신문','').replace('본 은 내용과 무관합니다','').replace('저작권자 오마이 시민기자 덧붙이는 글 이 글은 호주 동포신문 한호일보 에도 연재하고 있습니다','')
        text = text.replace('로그인 후에 작성하실 수 있습니다','').replace('원문 제충만의 페이스북','').replace('청주 연합 확 달라진 연합 웹을 만나보세요 네이버 연합 채널 구독 뭐 하고 놀까? 흥.','').replace('비즈니스워치 를 네이버 메인에서 만나요 비즈니스워치 스탠드 구독하기','').replace('이 에 하고 싶은 말을 남겨주세요','').replace('총 0개의 댓글이 있습니다','')
        text = text.replace('l이주형 경제산업 코리아','').replace('촉 뮤직엔터테인먼트 웹데일리','').replace('조선일보 4일자 2면 ','').replace('한국경제매거진 한경 잡앤조이','').replace('로그인 후 이용해주세요','').replace('잡앤조이 로 로그인 후 이용해주세요','').replace('키를 설정해 주세요','').replace('댓글 등록 중 알 수 없는 오류가 발생하였습니다','')
        text = text.replace('댓글 삭제 전 비밀번호 체크','').replace('댓글 정렬 댓글 가져오기','').replace('작성자만 삭제할 수 있습니다','').replace('비밀댓글이 아니거나 댓글작성자이거나 관리자페이지 의견관리로 들어왔을경우','').replace('비밀번호 확인 후 다시 시도해 주세요','').replace('신경철 웹데일리 기자','').replace('이경호 공유경제신문 기자','')
        text = text.replace('상단 왼쪽부터 김지현 정운선 이동하 윤나무 이형훈','').replace('글작성 완료 후 실행됩니다','').replace('개인정보 입력창을 띄우는 함수를 호출해 주세요','').replace('고객사 회원 연동 고객사 아이디','').replace('함현선 웹데일리 기자','').replace('비즈 인터플레이 중앙중단','').replace('헤드라인 이 시각 주요와 만나보세요','').replace('뉴시스 빅데이터 주가시세표 바로가기 저작권자 공감언론 뉴시스통신사','')
        text = text.replace('이 는 언론사에서 정치 섹션으로 분류했습니다','').replace('크립토허브 해피펫 ! 1코리아','')
        text = text.replace('해당 댓글을 수정 삭제 시 필요합니다','').replace('댓글내용을 입력해주세요','').replace('비밀번호를 입력해주세요','').replace('환경과조경 무단전재 및 재배포를 금지합니다','').replace('리슨투더시티의 장소상실 내성천 구럼비 옥바라지 골목','')
        text = text.replace('최초 등록 후 데이터 삭제 하면 데이터가 없다','').replace('등록 하시겠습니까?','').replace('저작권자 컨슈머타임스','').replace('파리 손진석 조선일보 특파원','')
        text = text.replace('구소장 저작권자 아시아타임즈','').replace('무단전재재배포 금지 댓글 띄어 쓰기를 포함하여 0자 이내로 써주세요','').replace('댓글은 글자 이상 쓰셔야 합니다','').replace('본문 삽입','').replace('댓글은 글자 이하로 쓰셔야 합니다','').replace('댓글을 입력하여 주십시오','').replace('이름이 입력되지 않았습니다','').replace('이 댓글을 삭제하시겠습니까','')
        text  = text.replace('따라서 네이버 의 ','').replace('의 배열 영역과','').replace('많이 본 댓글 많은 등과 같은','').replace('영역에는 반영되지 않습니다','').replace('이 자료는 언론사의 보도 가 아닌 각 기관에서 제공한 알림자료 입니다','').replace(' 메인으로추천 ','').replace('추천 수가 0인 경우 디자인이 달라서 클래스 추가 ','').replace('버튼 클릭후 호출되는 콜백함수','').replace('공유하기 레이어 노출 후 호출되는 콜백함수','').replace('공유하기 레이어 닫기 후 호출되는 콜백함수','').replace('l이주형 사회문화 코리아','').replace('건전한 토론문화를 위해 타인에게 불쾌감을 주는 욕설 비방 허위 명예훼손 도배 등의 댓글은 표시가 제한됩니다','')
        text = text.replace('조성경기자','').replace('나눔고딕 폰트가 설치되어있지 않습니다','').replace('국민방송 케이블방송 위성방송','').replace('양혜정 공유경제신문 기자','').replace('이병학 빅데이터 기자','')
        text = text.replace('글 입력','').replace('저작권자 오마이 시민기자 덧붙이는 글 ','').replace('이 는 의 개인 블로그 브런치에도 게재됩니다','').replace('김정후 빅데이터 기자','')
        text = text.replace('제공 롯데백화점 광주점','').replace('다문화가족 모국방문 항공권 전달식 경북도 제공','').replace('인터플레이 중앙중단','').replace('김성준 전 앵커 연합','').replace('비즈 인터플레이 중앙중단','').replace('면 좋아요','').replace('고은빛 한경닷컴 기자','').replace('l 뷰티풀 엑스','').replace('청와대 국민청원 게시판 캡처','').replace('디지털데일리 이대호기자','')
        text = text.replace('조경욱 웹데일리 기자','').replace('c 송수민기자','').replace('르꼬끄스포르티브 제공','').replace('저작권법에 의거 상업적 목적의 무단 전재 복사 배포 금지 ','').replace('은 해당 영상 캡처','').replace('미아방지 캠페인 포스터 아라리오 제공','').replace('조선의열단 0주년 기념사업추진위원회 제공','').replace('은 당시 보도된 중앙일보','').replace('대한민국 직장인 누구나 직장갑질 감수성 지수 테스트 페이지','').replace('에 들어가시면 자신의 감수성 점수를 확인할 수 있다','').replace('아람한의원 해운대점 윤다원 원장','').replace('인라이플 본문 삽입','')
        text = text.replace('데일리 금융경제','').replace('포항북부경찰서 제공','').replace('서울 영등포경찰서 로고','').replace('제회 고양국제대회 축구교류전 포스터','').replace('제공 고양시','').replace('베트남 여성 폭행 유튜브 캡처','').replace('진주 뉴시스 경남 진주시보건소','').replace('박재준 공유경제신문 기자','').replace('연합 아시아경제 김가연 인턴기자','').replace('한국경제 박찬휘 캐스터 글로벌 이슈 하이라이트','').replace('글 권세진 월간조선 기자','').replace('는 언론사에서 섹션분류를 하지 않았습니다','').replace('서울 연합 확 달라진 연합','').replace('웹을 만나보세요 네이버 연합 채널 구독 뭐 하고 놀까? 흥','').replace('본 는 헤럴드경제로부터 제공받은 입니다','')
        text = text.replace('제공 서울 연합','').replace('왼쪽부터 월드비전 양호승 회장 이화여자대학교 김은미 원장 월드비전 제공 공유경제신문','')
        text = text.replace('김성준 앵커','').replace('사공성근입니다','').replace('스포츠서울 조효정기자','').replace('저작권자 오마이 시민기자 덧붙이는 글 이 는 이정희 시민기자의 개인 블로그','').replace('와 미디어스 에도 실립니다','').replace('유튜브 l e ','').replace('포털사이트에서 유튜브 영상이 노출되지 않을 수 있습니다','').replace('저작권은 S에 있습니다','').replace('스트레이트 캡처','').replace('배열 영역과 ','').replace('직장갑질 감수성 지수 등급 평가는 직장갑질 감수성 지수 테스트 페이지','').replace('영화 뮬란 예고편 캡처','').replace('에서 누구나 할 수 있다','')
        text = text.replace('스포츠서울 윤소윤기자','').replace('이미지 연합 아시아경제','').replace('제공 유튜브','').replace('B저널','').replace('제공 송갑석 의원실','').replace('머니투데이 조해람 인턴기자','').replace('라디오 김성준의 시사 전망대 캡쳐','').replace('씨앤비저널 문화경제','')
        text = text.replace('엑스포츠 정아현 인턴기자','').replace('e g s d r ','').replace('sn m g g c n','').replace('g g s a','').replace('채널A','').replace('최주현 입니다','').replace('다음은 일자 미리 보는 이데일리 신문이다','').replace('중기부 제공 이데일리','').replace('건강가정지원센터 제공 이데일리','').replace('중기중앙회 제공 이데일리','').replace('자료 충북','').replace('청주S','').replace('온라인 커뮤니티 서울 뉴시스','').replace('이 는 언론사에서 사회 생활 섹션으로 분류했습니다','').replace('서울신문 김세연','').replace('대구 뉴시스 박준 기자','').replace('미국 뉴저지 언론 닷컴','').replace('코레일 강원본부 제공','').replace(' 제공 페르노리카 코리아','').replace('자료 이미지','').replace(' 채널A 한국금융신문','').replace('자동등록방지용 코드가 올바르지 않습니다','')
        text = text.replace('제공 영암군 영암','').replace('제공 의왕시 의왕','').replace('헤럴드경제 서병기 선임기자','').replace('스타 채준 기자 제공','').replace('제공 산청군','').replace('제공 동양생명','').replace('머니투데이 이호길 인턴기자','').replace('이진용 멕시코시티 기자','').replace('헤럴드P 천윤혜기자','').replace('추천 반대','').replace('김수아 빅데이터 기자','').replace('이미지 제공 제이엔지코리아 빅데이터','').replace('페이스북 로고 ','')
        text = text.replace('강경윤 기자','').replace('서울시 제공','').replace('제공 아시아경제','').replace('마마무 공식 한국금융신문','')
        text = text.replace('한국농어촌공사 제공','').replace('의 오승윤 왼쪽','').replace('조선생존기','').replace('커버 이미지','').replace('스포츠서울 이용수기자','').replace('제공 티앤아이컬쳐스','').replace('제공 아디다스글로벌 리딩','').replace('제공 전북','').replace('디지털데일리 백지영기자','').replace('영상 캡처 서울경제','').replace('안동 웅부공원과 문화공원 안동시 제공','').replace('이 는 김도헌 시민기자의 개인 블로그 브런치','').replace('미디어펜 석명 기자','').replace('유튜브 캡처','').replace('엔 캡처','')
        text = text.replace('티엔아이컬쳐스 제공','').replace(' 오현경 인턴기자','').replace('강지환 구속영장 신청 C캡처','').replace('하리수 인스타그램 캡쳐','').replace('는 언론사에서 경제 섹션으로 분류했습니다','').replace('한국당 울산 청년의원협의회 호프 미팅 자유한국당 울산시당 제공','').replace('좌 온라인 커뮤니티 우','').replace('서울메이트제공','').replace('이미지 리얼미터 공유경제신문','').replace('쇼셜미디어 캡처','').replace(' 캡처 로이터 연합','').replace('월트디즈니컴퍼니코리아 제공','').replace('캡처 서울 연합','')
        text = text.replace('유태훈 빅데이터 기자','').replace('자료 서울 연합','').replace('는 김상화 시민기자의 개인 블로그 ','').replace('에도 수록되는 글 입니다','').replace('프로듀스 X방송화면','')
        text = text.replace('외부 필자의 글은 본지 편집 방향과 일치하지 않을 수 있습니다','').replace('서지석 인스타그램 화면캡처','').replace('광주 한산 기자','').replace('정부세종청사 국세청 뉴데일리','').replace('아시아경제 뉴욕 김봉수 특파원','').replace('동아닷컴연기자','').replace('일간스포츠 설명','').replace('제공 현대차 스포츠서울','').replace('이미지 연합','').replace('생활건강 제공 넥스트 후','').replace('일간스포츠 서지영','').replace('김수산 리포터 저작권자','').replace('무단복제재배포 금지','').replace('네이버 홈에서 C 채널 구독하기','')
        text = text.replace('오중일 빅데이터 기자','').replace('제공 이데일리 스타','').replace('제공 스타벅스','').replace('제공 N','').replace('제공 신세계백화점','').replace('데이터저널리즘의 중심 데이터','')
        text = text.replace('유튜브 가로세로연구소 영상 캡처','').replace('김수연 웹데일리 기자','').replace('스포츠서울 이주상기자','').replace('텐아시아 제공','').replace('제공 광명시','').replace('대구 중부경찰서 제공','').replace(' 헤럴드경제 스타 컬처팀','').replace('스타 서울중앙지방법원 ','').replace('제공 양주시 ','').replace('머니투데이 중기 창업팀','')
        text = text.replace('스타 이소연 월드에디터','').replace('공유경제신문','').replace('제공 유니버설발레단','').replace('날카로운 통찰과 통통 튀는 생동감으로 가득차 있는칼럼','').replace('해시태그 청년 이 매주 화요일 오마이 독자를 찾아갑니다','').replace('머니투데이 뉴욕 이상배 특파원','')
        text = text.replace('저스티스 하이라이트 영상 캡처','').replace('제공ㅣS 저스티스','').replace('본문 텍스트 크게본문 텍스트 작게','').replace('박성준 빅데이터 기자','')
        
        text = text.replace('영상캡쳐','').replace('이소연 인턴기자','').replace('대구지방법원 전경','').replace('농촌진흥청 제공','')
        text = text.replace('픽사베이 이미지','').replace('김지현 인턴기자','').replace('오페라 피노키오의 한 장면.','').replace('임영호 한국농축산연합회 7대 회장.','').replace('한국화훼협회 제공','')
        text = text.replace('방어진지구대 소속 강중철 경사 왼쪽 구교성 순경.','').replace('울산동부서 제공','').replace('백지현 수습기자','').replace('빅이슈 캡처','').replace('편집자주','')
        text = text.replace('유튜브 영상 캡처','').replace('서울 북부지방법원촬영 이도연.','').replace('김효원기자','').replace('강주헌 박선영 인턴 기자','')
        text = text.replace('? 임신중감염 임신중감염 .','').replace('베이비는 창간 때부터 클린광고 정책을 유지하고 있습니다.','').replace('이것은 작은 언론으로서 쉬운 선택은 아니었습니다.','').replace(' 그럼에도 불구하고 베이비는 앞으로도 읽는데 불편한 광고는 싣지 않겠습니다.','').replace('베이비는 아이 낳고 기르기 좋은 세상을 만드는 대안언론입니다.','')
        text = text.replace('저희 가 마음에 드셨다면 좋은 후원하기에 동참해주세요.','').replace('여러분의 후원 참여는 아름다운 나비효과를 만들 것입니다.','').replace('베이비 좋은 후원하기 .','').replace('소중한 후원금은 더 좋은 를 만드는데 쓰겠습니다.','')
        text = text.replace('이철우 인스타그램','').replace('저작권자 경북일보 굿데이 굿','').replace(' 온라인팀 온라인팀의 다른 보기','').replace('c 뉴미디어국장 입니다.','').replace('인터넷신문과 영상 등 미디어 분야를 맡고 있습니다.','')
        text = text.replace('도쿄 P연합','').replace('헤럴드경제 모바일섹션','').replace('S노컷','').replace('와 관련 없는 자료 S쌈 마이웨이','').replace('오마이 글 김민준 편집 김준수','')
        text = text.replace('정민승 한국일보 호찌민특파원','').replace('이진송 계간홀로 편집장.','').replace('홍승한기자','').replace('이터널저니 예술은공유다 제공 ','')
        text = text.replace('한국경제 디지털이슈팀 ','').replace('권성진 인턴기자','').replace('그래픽 한국보건사회연구원 제공 .','').replace('이은 기자 .','')
    
        text = text.replace('스타투데이 전한슬 객원기자','').replace('내용 인용시 김지윤의 이브닝쇼 와의 인터뷰 내용임을 밝혀주시기 바랍니다.','').replace('내용 인용시 김지윤의 이브닝쇼 와의 인터뷰 내용임을 밝혀주시기 바랍니다.','')
        text = text.replace('굴림 i n','').replace('?패션엔 포토 강세정 셔츠와 청바지만 입어도 여신 포인트는 장화','').replace('?세정 동춘상회 취향 담긴 일상품 온라인 펀딩 잇따라 히트 .','').replace('?세정 관리부문장에 이훈규 부사장 영입 .','').replace('?첫 솔로 앨범 세정 스트라이프 매니시룩 우아한 걸크러시 .','').replace('?일리앤 신예은 차정원과 함께 5월 기프트 하트 컬렉션 전개 .','')
        text = text.replace('카테고리 넘버는 메뉴별로','').replace('김태현 다른보기','').replace('앵커 똑! 기자 꿀! 정보 시간입니다.','').replace('뉴욕 핌 민지현 특파원','').replace('서브가서브가면역관리 웹심포지엄 사전신청 면역관리 웹심포지엄 사전신청가','').replace('서브가면역관리 웹심포지엄 사전신청','')
        text = text.replace('?p','').replace('영상 제보받습니다','').replace('공감언론 뉴시스가 독자 여러분의 소중한 제보를 기다립니다.','').replace('가치나 화제성이 있다고 판단되는 또는 영상을 뉴시스 영상부 n .로 보내주시면 적극 반영하겠습니다.','')
        text = text.replace('t n i u i','').replace('투데이 앵커 마지막 소식 보시죠.','').replace('민지유 작 1편에서 계속 .','').replace('이 보도자료와 관련하여 보다 자세한 내용이나 취재를 원하시면행복청 고객지원팀 박단비 주무관에게 연락주시기 바랍니다.','')
        text = text.replace('글자크기 작게하기 가','').replace('글자크기 크게하기 가','').replace('서울 성소의 인턴기자 조민교 인턴기자','').replace('이 기자의 를 구독하시려면 구독 신청 버튼을 눌러주세요.','').replace('구독할수 없는 기자 입니다.','')
        text = text.replace('이미지 제공 오비맥주 빅데이터','').replace('윤용민 기자','').replace('아이즈 바로가기 스타들의 다양한 을 더 크게!','').replace('포토 갤러리 아이즈 e의 스타인터뷰 아이즈 와 사전협의 없는 무단 도용 전재 및 복제 배포를 금합니다.','')
        text = text.replace('이 시각 누리꾼들의 관심을 받고 있는 는 어떤 게 있을까요.','').replace('라이브투데이 핫클릭 첫 소식부터 살펴보겠습니다.','').replace('제공 에프이스토어','').replace('자료 영상 화면 캡처','')
        text = text.replace('투데이 앵커 다음 소식 보시죠.','').replace('본 는 머니S 제8호 년 월일 에 실린 입니다.','').replace('c 모바일 오픈! 미술작품 가격을 공개합니다.','').replace('박재우 기자','')
        text = text.replace('온라인 의 강자 머니투데이가 그 날의 가장 뜨거웠던 이슈를 선정해 다양한 각도에서 조명해드립니다.','').replace('어떤 이슈들이 온라인 세상을 달구고 있는지 이슈 를 통해 확인하시기 바랍니다.','')
        text = text.replace('이데일리 다음은 일자 이데일리 신문 주요 다.','').replace('이 글 자체가 스포일러입니다.','').replace('N이 선정한 이 한 장의 입니다.','').replace('서울 핌 s i r n n l','').replace('의오렌지블론드아메리카노이미지','')
        text = text.replace('지금까지 더블클릭이었습니다.','').replace('아이즈 글 김리은 임현경 김현민 .','')
        
        text = text.replace('에스엔에스 빅데이터','').replace('o 에 오슬로 평화연구소와 오슬로대학의 연구진이 기고한 . ? a ? 를 번역한 글입니다.','').replace('제공 촉뮤직엔터테인먼트','').replace('공주향토민요발전방안토론회포스터.','').replace('? 카카오스토리','')
        text = text.replace('경향신문 유튜브 갈무리.','').replace('김용명 인스타그램','').replace('자료 하나금융투자 ','').replace('스타 주성배 인턴기자','').replace('조세금융신문 내 박스','')
        text = text.replace('전체 방송 중 내용 일부만을 담았습니다.','').replace('어려운 용어나 표현 등은 의미가 달라지지 않는 범위에서 알기 쉽게 수정했습니다.','').replace('조선 원더풀데이 캡처','').replace('정단비 인턴','').replace('광주고등법원 전경','')
        text = text.replace('네이버 연합 채널 구독','').replace('이에스더 복지행정팀 기자','').replace('m유재형 기자','').replace('코멘트를 등록하기 위해서는 로그인이 필요합니다.','').replace('엑스포츠 하선영 인턴기자','').replace('머니투데이 서민술','').replace('미쓰백 포스터 제공 리틀빅픽쳐스','')
        text = text.replace('의 이해를 돕기 위한 자료','').replace('공공 공정 공감 가치 실현 헤럴드경제','').replace('이소정 기자 소통하는 제공','').replace('오늘은 와 함께합니다.','').replace('g팔로우하기 e e 좋아요 누르기','').replace('편집자 주I보고서 전문 및 쌍방향 지도','')
        text = text.replace('내용 인용시 김어준의 공장 과의 인터뷰 내용임을 밝혀주시기 바랍니다.','').replace('한국경제 지수희입니다.','').replace('버튼 태그 골라 사용','').replace('투데이 앵커 이어서 오늘 아침신문 보겠습니다.','')
        text = text.replace('이런데도 각본 없이 했다고? ㅋㅋㅋ','').replace('?남영비비안 신임 대표이사에 손영섭 부사장 선임','').replace('?남영비비안 스킨핏 누디 패키지 인기 만점 목표 대비판매','')
        text = text.replace('?남영비비안 쌍방울 한국야구위원회 와 콜라보 마스크 제작','').replace('?비비안 시원한 쿨링감의 초여름 기능성 홈웨어 출시','').replace('?남영비비안 편의점 에서 애슬레저 레깅스 판매','')
        text = text.replace('지난6월지속가능발전주민공청회장','').replace('엔터테인먼트 이데일리 스타i','').replace('이 곳에 설명을 써주세요.','').replace('아이즈 글 김소미 씨네기자','')
        text = text.replace('국민리포트 김용옥입니다.','').replace('앵커 클릭 지구촌 입니다.','').replace('투데이 앵커 마지막 키워드 볼까요?','').replace('투데이 앵커 밤사이 인터넷과 S를 달군 따끈따끈한 화제의 이슈를 전해드리는 투데이 이슈 톡 시간입니다.','')
        
        text = text.replace('앵커 오늘 소식은 여기까지입니다.','').replace('이벤트 한국일보 채널 구독 추천하면 에어팟이 내꺼! 뉴 잼을 보면 가 재밌다.','').replace('이버에서 매일신문 구독하기','')
        text = text.replace('매일신문 네이버 바로가기','').replace('이 는 언론사에서 경제 사회 생활 섹션으로 분류했습니다.','').replace('리뷰 시사 직격','').replace('제공 신한은행 빅데이터','')
        text = text.replace('애플 한국에서만 아이폰E 모델 출시 안 한다 .','').replace('업계에 따르면 애플이 한국 이통사의 의견을 수용해 국내에 모델만 출시한다.','').replace('나이키 아디다스 반스 신상 최대세일 시작한 스포츠 지금 안 사면 후회한다','')
        text = text.replace('스포츠가 신상품을 최대할인하는 여름 시즌 세일 을 시작한다','').replace('년 총선의 선거운동기간 동안','').replace('게시판에 특정 후보자들의 명예훼손 비방 악의적 댓글과 불법적인 선거홍보를 할 경우 공직선거법 제0조 후보자 등의 비방금지 등의 위반으로 민형사상 피해가 일어날 수 있습니다.','')
        text = text.replace('게시자분들의 예기치 않은 피해를 방지하고자 아래 선거운동기간 동안 게시판의 운영을 잠정 중단하오니 양해해주시기 바랍니다.','').replace('관련 동영상 보기','').replace('회원이시라면 로그인후 이용해보십시오','')
        text = text.replace('비회원은 댓글쓰기 권한이 없습니다','').replace('지금까지 비디오 디저트였습니다.','').replace('박람회 대구국제안경전 일 개막','').replace('세미나 소재 트렌드 세미나','').replace('전시 패션 작가 보리 1주기 추모전','').replace('모집 잠뱅이 청바지 패션블로거 체험하다','')
        text = text.replace('월트디즈니 컴퍼니 코리아 제공','').replace('국민일보 홈페이지에서 확인 가능합니다.','').replace('라이온봇 기자 한국경제 본 는.','')
        text = text.replace(' 금융 전문기업 씽크풀이 공동 개발한 자동생성 알고리즘에 의해 실시간으로 작성된 것입니다','').replace('예작 일본 상표권 출원','').replace('글로벌 공략 나선다','')
        text = text.replace('언택트 바람에 1분기 온라인 매출 전년대비증가','').replace('온라인 전용 여성 브랜드 본이 런칭','').replace('형지에스콰이아 형지아이앤씨 흑자전환 성공','').replace('내실있는 성장 다져','').replace('본 아프리카와 예술의 접목 매그넘 컬렉션 런칭','').replace('?형지I','')
        text = text.replace('확 달라진 연합 웹을 만나보세요','').replace('뭐 하고 놀까? 흥','').replace('나눔고딕','').replace('u e','').replace('지금까지 오늘의 픽 오픽이었습니다.','').replace('자기랑 똑닮은 생후 1년 아들 최초 공개한 초보 아빠 유재명','').replace('배우 유재명이 아들 모든이를 최초로 공개해 누리꾼의 관심을 집중시켰다','')
        text = text.replace('신혼가전 우리 집 크기에 잘 맞을까? 지금도 앞으로도 우리 집에 꼭 맞는 삼성 신혼가전','').replace('삼성 신혼가전이 이사와 출산에도 걱정할 필요 없이 집안을 꾸밀 수 있는 다양한 가전을 준비했다','')
        
        text = text.replace('유튜브 화면 캡처','').replace('이승우 지음 김영사 펴냄 1만원 본 는 머니S 제2호에 실린 입니다','').replace('영생보감의 서양탕국 ','').replace(' 광고 캡처','').replace('배우 모델 구인 카페에 올라온 구인 글','').replace('제공 시흥시','').replace('제공 여성가족부','')
        text = text.replace('스포츠서울 김민규기자','').replace('제공 생활건강생활건강','').replace('I대구일보 모바일내중하단','').replace('I대구일보 모바일내상단','').replace('군프롭테크 부동산을 부탁해!','').replace('우리 아기가 태어났어요','').replace('역시 의사선생님의 글이 신뢰가 갑니다','').replace('제공 이마트','').replace('이하 캡처','').replace('일간스포츠 권지예','')
        text = text.replace('스타케이 영상 캡처 ','').replace('제공 나무액터스','').replace('문의 광주독립영화관 5황해윤 기자','').replace('섹션 연예통신 방송화면','').replace('남순 유튜브 캡쳐','').replace('프로듀스X방송 화면','')
        text = text.replace('충주 티팬티남 커뮤니티 캡처','').replace('수출무역관리령 링크','').replace('상생파트너 ㅣ 스타트업','').replace('월트디즈니 코리아 컴퍼니 제공','').replace('한국일보 자료','').replace('제공 신라면세점','').replace('대구시 남구청 제공','').replace(' 서울청 소속 기동대원 상대동의 없이 동영상 촬영 피소 자료','').replace('그래픽 유정수 디자이너','').replace('M경기방송 강인묵기자','').replace('창원 뉴시스 김기진','').replace('조성복 웹데일리 기자','').replace('그림 김희서 인턴기자','').replace('온라인 커뮤니티 캡처','').replace('제공 코오롱C','').replace('제공 각 배급사','').replace('m 열린마당 신앙인의 눈','')
        text = text.replace('충북국제무예액션영화제 포스터 교체전 왼쪽 과 교체 후 이미지','').replace('제공 엔터테인먼트가수','').replace('제공 롯데홈쇼핑','').replace('제공 소울시즌스 스포츠서울','').replace('정기구독 한국경제매거진 한경비즈니스 당사의 허락 없이 본 글과 의 무단 전재 및 재배포를 금합니다','').replace('n이제 마이페이지에서 구독 하실 수 있습니다','').replace('기자의 를 구독 하시겠습니까?','').replace('구독이 완료 되었습니다.','').replace('에 접속한 뒤 메인화면의 빅데이터 ','')
        text = text.replace('한편 년 8월1일부터 선보인','').replace('한국이주여성연합회 제공','').replace('빅데이터I는 투자 참고용이며 투자 결과에 책임을 지지 않습니다','').replace('를 클릭해 이용하면 된다','').replace(' 구글플레이 또는 애플 앱스토어에서 뉴시스 앱을 내려받거나 뉴시스 모바일 홈페이지 m ','')
        text = text.replace('난설 공연장면 중 배우 유승현 왼쪽 하현지 자료','').replace('난설 공연장면 중 배우 유승현 자료','').replace('난설 공연장면 중 배우 유현석 자료','')
        text = text.replace('난설 공연장면 중 배우 정인지 왼쪽 백기범 자료','').replace('난설 공연장면 중 배우 정인지 자료','').replace('난설 공연장면 중 배우 안재영 자료','').replace('난설 공연장면 중 배우 안재영 자료','').replace('난설 공연장면 중 배우 하현지 자료','')
        text = text.replace('도착 항구를 검색 해주세요','').replace('항구 이름으로 검색해서 목록 가져오기','').replace('데이터를 가져오는데 문제가 있습니다','').replace('선박운항스케줄목록으로 e t t t를 나타내는 테이블입니다','').replace('선박운항스케줄 목록','').replace('서울신문 나우 자료 f','').replace('이미지 제공 ','').replace('트라이씨클 빅데이터','').replace('산청군 제공 경호강 래프팅','').replace('제공 알렉산더 맥퀸','').replace('제공 한자원 한국자격연구원','')
        text = text.replace('보도자료','').replace('중앙포토','').replace('난설 공연장면 중 배우 하현지 왼쪽 유현석 자료','').replace('난설 공연장면 중 배우 백기범 자료','').replace('제주교통복지신문 편집부','').replace('일간스포츠 권오용','').replace('S제공','').replace('구독 온라인 서비스 신청','').replace('로그인 후 작성 가능합니다','').replace('댓글을 수정하시겠습니까?','').replace('답글을 수정하시겠습니까?','').replace('글을 삭제하시겠습니까?','').replace('잠시만 기다려 주십시오','')
        text = text.replace('제공 국립현대미술관','').replace('정말 삭제하시겠습니까?','').replace('자료 리얼미터','').replace('이데일리','').replace('제공 K쇼핑','').replace('의정부 파이낸셜','').replace('제공 경기북부청','').replace('제공 하나금융','').replace('제공 우리금융','')
        text = text.replace('스타 최유진 인턴기자','').replace('연예가중계 캡처','').replace('스포츠서울 이게은기자','').replace('책과함께 제공','').replace('헤럴드P 김나율기자','').replace('와 관련 없는 자료','').replace('제공 기아자동차','').replace('일간스포츠 이승한기자','').replace('제공 Y엔터테인먼트','').replace('제공 곡성군','')
        text = text.replace('글 인턴 김성은 감수 진행 출연 라미 안군 씬디','').replace('로 질문과 사연 보내주세요','').replace('M박동선기자','').replace('제공 천안서북경찰서','').replace('김필주 웹데일리 기자','').replace('보배드림 캡처','').replace('제공 C엔터테인먼트','').replace('제공 나일론','').replace('제공 멜로가 체질','').replace('제공 S','').replace('골룸 골라듣는 룸 팟캐스트는 팟빵 이나 아이튠즈 에서도 들을 수 있습니다','')
        text = text.replace('제공 한세실업','').replace('연합 제공','').replace('제공 올리브네트웍스 올리브영 빅데이터','').replace('남부경찰서 제공 부산','').replace('제공 월트 디즈니 컴퍼니 코리아','').replace('제공 다이어리알','')
        text = text.replace('와 무관한 이미지 입니다','').replace('기호일보 디지털부','').replace('제공 대한자동차경주협회','').replace('제공 롯데홈쇼핑 빅데이터','').replace('스포츠서울 글 성남 이주상기자','').replace('인제군 제공','').replace('언론지원단 빅데이터','').replace('제공 월트디즈니컴퍼니코리아 서울경제','').replace('시나닷컴 캡처','').replace('경남지방경찰청 연합 자료','').replace('유튜브 캡처','')
        text = text.replace('메디게이트 임솔 기자','').replace('이은지 수습기자','').replace('박하얀','').replace('제공 매거진 앳스타일','').replace('제공 수원시','').replace('자료 한국노동연구원 아시아경제','').replace('스포츠서울 조윤형기자','').replace('최수정 인스타그램','').replace('마 장민혜 객원기자','').replace('제공 엔터테인먼트','').replace('김나희 앳스타일','')
        text = text.replace('제공 유한킴벌리','').replace('저작권자 소비자가 만드는 신문','').replace('지원액은 월 1만0원 연 만원까지 받을 수 있다','').replace('디지털데일리 최민지기자','').replace('서울 연합 이웅 기자 ','').replace('참고문헌 사베리오 가에타 교황 프란치스코','').replace('새 시대의 응답자 강선남 옮김 성바오로','').replace('매튜 번슨 교황 프란치스코 그는 누구인가 제병영 옮김 하양인','').replace('호르헤 마리오 베르고글리오 아브라함 스코르카 천국과 지상 옮긴이 강신규 율리시즈 ','')
        text = text.replace('서울프린지페스티벌티저','').replace('인터뷰 송유경 대구 달서구','').replace('이성규 과학팀 기자','').replace('P의 압력을 종아리는','').replace('P의 압력 허벅지는','').replace('P의 압력 엉덩이는','').replace('가족행복플라자 구미시 제공','').replace('제공 우먼센스','').replace('제공 경기도','').replace('제공 카페띠아모','').replace('제공 일차돌차돌박이전문점','').replace('제공 부엉이돈까스','').replace('제시 주 인스타그램 캡쳐','')
        text = text.replace('제공 파주시','').replace('일간스포츠 제공','').replace('본문과 관련없는 이미지','').replace('제공 포천시','').replace('은 특정 와 상관 없음','').replace('제공 으라차차','').replace('너 때문에 채팅창 난리났다 이 형 지상파는 아닌 듯','').replace('삭제된 메시지입니다 채팅 예절을 준수해주세요','')
        text = text.replace('가수 서문탁 서문탁 인스타그램','').replace('시팀 기자','').replace('정재영의 이슈i 개인정보ㆍ전범기업','').replace('에필로그리카도 인스타그램','').replace('에필로그리카도의 인스타그램 프로필','').replace('제공 롯데백화점 빅데이터','').replace('제공 A생명','').replace('비바시티 모델 강민경 빙그레 제공','').replace('자료 김규종 인스타그램','')
        text = text.replace('한국경제 종목알파고봇 기자 ','').replace('제공 한국공인회계사회 ','').replace('제공 자이글 스포츠서울','')
        text = text.replace('서브가한국 팜비오 오라팡 광고 ','').replace('모바일서브다약사랑생명사랑','').replace('제공 포항시청','').replace('정지원 빅데이터 기자','').replace('박하선이 착용한 지고트의 린넨 원피스 상품 바로가기 패션엔 이민지','').replace('스포츠서울 김선우기자','').replace('한국경제 박준식입니다','').replace('유튜브 채널 캡처','').replace('제공 주 노랑풍선','').replace('캡처 헤럴드경제 정지은 인턴기자 ','').replace('머니투데이 이은 기자 ','').replace('광주S 자료','').replace(' 특정표현과 관계 없음','').replace('특정표현과 관계없음','').replace('대덕구 제공','')
        text = text.replace('관련 방금 보신 와 같이 읽으!','').replace('인삼공사 여름철 홍삼 정관장 아이패스 프로모션 진행 정관장 아이패스 제품 구매 시 다양한 해택 제공','').replace('강남구 제공','').replace('양구 시래기 솥밥 시래기 전골 레시피 북 이미지','').replace('제공 자라','').replace('무단 전재 재배포 금지','').replace('충남 부여경찰서 연합 자료','').replace('기고 김미순 제주시 애월읍사무소','').replace('김미순 제주시 애월읍사무소','').replace('자료 국토교통부','').replace('최선의 나비 와 침대성 수원아이파크시립미술관 제공','').replace('m에서 이 게시물 보기','').replace('s e o . a k a o e r a d d e e l k e n . e k e 님의 공유 게시물님7월오후','')
        text = text.replace('블룸스틱 제공','').replace('민병진 의 전체 보기','').replace('자료제공 와이즈앱 와이즈리테일','').replace('제공 창원서부경찰서','').replace('엑스포츠 이소진 인턴기자','').replace('영상 설명 ','').replace('n 영상 스케치','').replace('제공 오산시','').replace('제공 한국에너지재단','')
        text = text.replace('제공 키이스트','').replace('제공 경산시 ','').replace('제공 리테일 빅데이터 ','').replace('제공 초록뱀미디어C','').replace('o 캡처','').replace('제공 군포시','').replace('광주시교육청 광주시교육청 제공','').replace('해당 언론사가 주요로직접 선정한 입니다','').replace('광주S 김삼헌기자','').replace('자료 광주광역시교육청 ','').replace('제공 경기남부청','').replace('서울 김수경 에디터','').replace('ㅣ한국저작권협회 캡처','').replace('제공 살림하는 남자들','').replace('취재기자 이지현 방송일조회','').replace('제공 경주시청','').replace('제공 서울','').replace('리얼미터오마이 현안조사 ','').replace(' 브룸스틱 제공','').replace(' 캡쳐 하트시그널','').replace('여성가족부 제공','')
        text = text.replace('제공 호텔신라','').replace('제공 M','').replace('꼭 필요한 경제정보만 모았습니다 비즈니스워치 네이버 포스트 구독하기','').replace('당신이 바빠서 흘린 이슈 줍줍이 주워드려요 레터 줍줍 구독하기','').replace('카카오게임즈 제공','').replace('와이티엔화면','').replace('제공경남','').replace('디지털데일리 윤상호기자','').replace('머니투데이 구단비 인턴','').replace('포스터 이미지 국립극단 제공','').replace('제공 과천시','').replace('지파운데이션 공식 홈페이지','')
        text = text.replace('이민준입니다.','').replace('강성웅입니다.','').replace('다음 웹툰 남남 화면 갈무리','').replace('터치 부산 앱 출시 부산일보 지면을 내 손 안에!','').replace(' 구혜선 인스타그램','')
        text = text.replace('광고영상 캡처 컴투스 제공','').replace('행사 이벤트 진행','').replace('웹데일리 취재 한예은 스웨덴 리포터 정리 ','').replace('인터뷰 이성한 연합 신임 편집총국장','')
        text = text.replace('모금 계좌 우리은행서울가톨릭사회복지회','').replace('이 는 독자 여러분들의 소중한 구독료로 제작되었습니다.','').replace('이 는 독자 여러분들의 소중한 구독료로 제작되었습니다.','')
        text = text.replace('선교용으로 신문을 후원하시려면 클릭해 주세요.','').replace('프린트는 유료회원이 되어야 사용할 수 있습니다.','').replace('스크랩은 유료회원이 되어야 사용할 수 있습니다.','')
        text = text.replace('외부 필진의 원고는 본지의 편집방향과 일치하지 않을 수 있습니다.','').replace('기획 특집 특집일반','').replace('방송일시 년 월 일 수요일','')
        text = text.replace('비지트인뉴욕 숏패딩 상품 바로가기','').replace('숏패딩 상품 바로가기 패션엔','').replace('비지트인뉴욕 특별한 일상봄 여름 광고 캠페인 공개','').replace('비지트인뉴욕 주체적인 여성 위한가을 겨울 광고 캠페인 공개','').replace('애드호크 내추럴 무드의 어스톤가을 겨울 캠페인 공개','').replace('애드호크 섬머시즌 데일리웨어로 제격! 시리즈 출시','')
        text = text.replace('애드호크 활기찬 봄 분위기 담은S 캠페인 화보 공개','').replace('인용보도 시 프로그램명 전용우의 을 밝혀주시기 바랍니다.','').replace('저작권은 에 있습니다.','')
        text = text.replace('오진영 인턴','').replace('조권이 지난 8일 오후 열린 뮤지컬 제이미 원제 프레스콜에 참석해 공연 넘버 r n 하이라이트 장면을 시연했습니다.','').replace('보이그룹 베리베리 가 오늘 1일 오후 열린 미니 4집 앨범 페이스 유 발매 기념 온라인 쇼케이스를 열고','')
        text = text.replace('최근 별세한 백선엽 장군이 국립대전현충원에 안장되는 것을 두고 국내에서 극심한 대립 양상이 펼쳐지자 미국 내 여론도 주목하는 분위기다.','').replace('김기현 미래통합당 의원은 이인영 통일부 장관 후보자 인사청문회 자료 제출을 거부하고 있는 점을 지적하며 인사청문회를 대놓고 무력화시킬 작정이냐 고 비판했','')
        text = text.replace('부동산 대책이 발표됐다.대책 이후 불과 3주 만이다.','').replace('정부가 집값 안정화 의지 를 다시 한번 피력한 셈이다.','').replace('당사의 허락 없이 본 글과 의 무단 전재 및 재배포를 금합니다.','')
        text = text.replace('서울디지털산업단지 연합 자료','').replace('본 이미지는 와 상관없는 자료입니다','').replace('제공 씨제스엔터테인먼트','').replace('르노삼성차 팬텀싱어우승팀 라포엠에 제공','').replace('제공 재 예술경영지원센터','').replace('제공 한국연합회','').replace('스포츠서울 이지석기자','').replace('제공 세포라','').replace('리얼미터 제공','').replace('김운성 작가 제공','').replace(' 내용과 관련이 없습니다','').replace('내용과 무관함','').replace('t o t a c y','').replace('현t ','').replace(' i 임상약학과 교수','').replace('제공 유니베라','').replace('헤럴드경제 이운자','').replace('아침신문 솎아보기','').replace(' 보도 캡처','').replace('부천 송지수 인턴기자','')
        text = text.replace('a t 공식 웹사이트','').replace(' a t 공식 인스타그램','').replace('강태오 황지은기자','').replace('한국엘러간 사 홈페이지','').replace('라이프타임 제공','').replace('특정부분에 데이타 넣기','').replace('서울 뉴시스 정리김태규 기자','').replace('은 와 관계 없습니다','').replace('성영주 코스모폴리탄 디지털디렉터','').replace('제공 영등위','').replace('이주형 사회문화 코리아','').replace('서울 뉴시스 정리김태규 기자','').replace('광주기독병원 노조 제공','').replace('영상M 베리베리 l 7인 완전체 버전 최초 공개','').replace('청와대 제공 재판매 및 금지','')
        text = text.replace('이향규 후아유 창비교육','').replace('다음은 일자 ','').replace('독자 여러분의 제보를 기다립니다','').replace('공현주 인스타그램','').replace('대한출판문화협회 제공','').replace('결정장애?','').replace('S스페셜 제공','').replace('저작권자 무등일보 제공','').replace('전화 0카카오톡 플러스친구 무등일보','').replace('광주 전남지역에서 일어나는 사건사고 교통정보 미담 등 소소한 이야기들까지 다양한 사연과 영상 등을 제보받습니다','').replace('김예나 인턴 기자','').replace('인터뷰 홍봉진기자','').replace('광주광역시 빅데이터','').replace('제주S 이인 기자','').replace('네티즌 찬반 갈려 더팩트ㅣ송은화 기자','').replace('찬딜 발로치 공식 페이스북','').replace('호텔롯데 제공','').replace('뉴 제공','').replace('ㅣ강영국 기자','').replace('김효정 에디터','').replace('이를 어길 시에는 민 형사상 법적책임을 질 수 있습니다','').replace('본 는 월간 통권 제2호 년 9월호 게재입니다','').replace('본 와 은 더뮤지컬 이 저작권을 소유하고 있으며 무단 도용 전재 및 복제 배포를 금지하고 있습니다','') 
        text = text.replace('이 에디터의 다른 글 보기','').replace('제공 신한금융그룹 신한희망재단 빅데이터','').replace('이 를 다른 회원들도 보면 좋겠어요!','').replace('공유하고 소중한 사람의 건강 지켜주세요!','').replace('주소 복사가 완료되었습니다. 를 눌러 원하는 곳에 붙여넣기 하세요','').replace('토스 앱 화면 캡처','').replace('서울경제 고현정입니다','').replace('방송된 화면 캡처','').replace('광주 서부경찰서 연합 자료 광주 연합','').replace('재판매 및 금지 울산 연합 ','').replace('a k a o e r a d d e e l k e n . e k e 님의 공유 게시물님7월오후','').replace('제공 이천시 신둔면 주민자치학습센터','').replace('주 슬기로운생활 서울특별시 강남구 봉은사로길 1호 역삼동','').replace('사업자등록번호인터넷신문등록번호 서울 아2','').replace('대표 정주식 발행인 정주식','')
        text = text.replace('외부 필진의 컨텐츠는 직썰의 편집방향과 다를 수 있습니다','').replace('국민청원 캡처','').replace('서울신문 베트남항공 자료','').replace('ㅣ신정환 인스타그램 스포티비','').replace('백지장도 맞들면 낫다 고 했다','').replace('쇼핑도 하고 선의에 동참하는 기쁨도 누려보고 싶다면? 지금부터 소개하는 협업을 눈여겨볼 것','').replace('스포츠서울 김대령기자','').replace('제공 인구보건복지협회','')    
        text = text.replace('투어타임즈 정기환기자','').replace('대한민국 일자리 으뜸기업 선정 노랑풍선 제공','').replace('아이즈 글 황효진 칼럼니스트','').replace('아이즈 글 송해나','').replace(' 우먼센스 제공','').replace('화이브라더스코리아 제공','').replace('인터넷 와글와글이었습니다','').replace('제공 자이글','').replace('모든 수정폼 숨기고 뷰폼 보이기','').replace('본 는 머니S 제3호 년 7월일 8월5일 에 실린 입니다','').replace(' 이바미 인턴','').replace('그래프 남영비비안 일봉 차트이 는 핌과 ','')
        text = text.replace('한경로보이 는 한국경제신문과','').replace('도움말 나사렛국제병원 재활의학과 정태호 과장 기호일보','').replace('드라마 장면 캡처','').replace('에스엔에스 웹데일리','').replace('서울신문 자료','').replace('서비스 이용은 임신육아종합포털 아이사랑','').replace('e 제공','').replace('산청군 지역사회 중심 재활사업 간담회 개최입력','').replace('곽상욱 오산시장 오산시','').replace('경남도청 자료','').replace('중앙일보헬스미디어 입력','').replace('이미지 M','').replace('생명과학 한경탐사봇이 는','').replace('한경닷컴과 버핏연구소가 공동 개발한 탐사봇 알고리즘에 기반해 작성된 것입니다','').replace('선정 열띤 토론입력','').replace('런던 권석하 재영칼럼니스트','').replace('국립대구박물관 제공','')
        text = text.replace('제공 페이스북','').replace('정열 기자','').replace('이번 연구결과는 국제 학술지 당뇨병 의학 i n 최근호에 게재됐다','').replace('의정 토크 경기포커스','').replace('경인방송   ','').replace('경인방송 방송 경인방송 라디오 박마루의 시사포차 년 월 일진행 ','').replace('제공 B하나은행 빅데이터','').replace('제공 나무엑터스O','').replace('초록벰미디어 제공','').replace('일 현지시간 통신과 현지언론 인덱스','')
        text = text.replace(' 시사포차 년 월 일진행 ','').replace('박마루 인터뷰 김강식 경기도의회 의원 박마루 본격','').replace('살림남 캡처','').replace('다음은 1일자  ','').replace('주요 다','').replace('취재기자 이재욱 방송일조회','').replace('제공 스포티비','')
        text = text.replace('머니투데이 송민경','').replace('제공 탑텐 ','').replace('한국금융신문 ','').replace('광주광역시 교육청사 시교육청 빅데이터','').replace('3일간 편 상영 오마이','').replace('제공l 스포티비','').replace('이번 연구 결과 a a t ir 는 아동 발달 저널 에 게재되었다','').replace('울산시 북구 제공','').replace('재판매 및 금지 울산 연합','').replace('예카테리나 카라글라노바 인스타그램 화면 캡처','').replace('전장x전폭x전고는I가 m다','').replace('웨이보 영상 캡처','').replace('매일경제 스타투데이 차윤주 인턴기자','').replace('이미지 오비맥주 제공','').replace('아시아경제 조슬기나 기자','').replace('배우 강성욱 강성욱 인스타그램','').replace('경주시 제공','').replace('t의 a ','').replace('를 번역한 글입니다','').replace('멜로가 체질 예고 영상','')
        text = text.replace('스타 채준 기자','').replace('제공 유리아쥬','').replace('성공회대 교수 수원시정연구원장','').replace('l손혁재 사회문화 아시아','').replace('역사속오늘 비키니 첫 등장고흐 자살팔레스타인 살 저항운동가 아헤드 타미미 석방법륜스님 막사이사이상','')
        text = text.replace('영상 캡처','').replace('!플레이어 스크립트 수정 시작 .','').replace('!플레이어 스크립트 수정 끝 .','').replace('? b이베이코리아 클라우드 데이터 개발자 채용','').replace('? b카카오스토리 . ','').replace('이베이코리아 클라우드 데이터 개발자 채용 .','')
        text = text.replace('페이지 본문내우측','').replace('양도세는까지 오른다 . 재계 김동관 김남호 정기선 주목 경영능력 한단계씩 업그레이드 . 최저임금 노사 간극0원 원 원 . 단통법 개선안 나온다.','').replace('판매장려금 에 쏠린 눈 . 은행권 2분기 실적 사모펀드가 판가름 . 과기부 중재 불구 딜라이브 사용료 논의 평행선 . 대전 코로나 n차감염 확산 일 2명 확진 판정 . 지역갈등 초래 X세종역 당초 계획대로 추진 . 대전 코로나 계속 확산 고강도 거리두기 일까지 연장 . 충청날씨 일 새벽부터 비 저녁 대부분 그쳐 . 충청브리핑 박원순 서울시장 극단선택 충격 안희정 오거돈 이어 미투 의혹 .','').replace('하단 메인 반복 모듈','')
        text = text.replace('제공 부산지방경찰청 ! ','').replace('! 0명이 을 를 좋아합니다.','')
        text = text.replace('이한결 인턴기자','').replace('연합 자료 테헤란 연합 강훈상 특파원','').replace('? 제공 그룹 빅데이터 .','').replace('기술보증기금 본점 전경.','').replace('웹툰 머니게임 스튜디오N 제공 .','')
        text = text.replace('밴디트 개인 티저 포토.','').replace('제공 H엔터테인먼트 .','').replace('청와대 홈페이지 화면 캡처 .','')
        text = text.replace('? d카카오스토리 .','').replace('제공 부산선관위 ! .','').replace('','').replace('','').replace('','')
        text = text.replace('제공 양평군 ','').replace('배구선수 김요한 인스타그램','').replace('요청하신 페이지를 찾을 수 없습니다','').replace('영국 옥스퍼드대출판사 어 베리 쇼트 인트로덕션 r o 시리즈','').replace('김성국 옮김 이학사 1만원','').replace('서재길 옮김 책과함께 1만원','').replace('박희진 옮김 솔 2만원','').replace('제공 연합 자료제공 국민건강보험공단','').replace('관람평도 이어져 오마이','').replace('김삼웅 지음 두레 2쪽 1만 원','').replace('김진호 외 3명 지음 철수와영희 4쪽 1만 원','').replace('연합 자료 로마 연합 전성훈 특파원','').replace('연합 자료 광주 연합','').replace('은 뉴욕주 성범죄자신상정보 사이트','').replace('정유미 인스타그램','').replace('제공 P엔터테인먼트','').replace('머니투데이 정명근 에디터','').replace('패션엔 이민지','').replace('김성령 인스타그램','').replace('반말과 비속어가 있더라도 원문에 충실하기 위해 그대로 인용합니다','').replace('커뮤니티 글 제목을 클릭하시면 원문을 볼 수 있습니다','').replace('두레ㆍ2쪽ㆍ1만0원 우주날씨 이야기황정아 지음','')
        text = text.replace('플루토ㆍ2쪽ㆍ1만0원 루카치가 읽은 솔제니친게오르크 루카치 지음','').replace('산지니ㆍ0쪽ㆍ1만0원 삶으로서의 사유게오르크 루카치 지음','').replace('','').replace('산지니ㆍ6쪽ㆍ3만원 나 시몬 베유시몬 베유 지음','').replace('솔과학ㆍ3쪽ㆍ3만0원 두 명은 아니지만 둘이 살아요김용운 지음','').replace('문학동네ㆍ8쪽ㆍ1만0원 아침 그리고 저녁욘 포세 지음','').replace('덴스토리ㆍ4쪽ㆍ1만0원 세기 국제정치와 투키디데스로버트','').replace('마카롱ㆍ4쪽ㆍ1만0원 반과거장승리 지음.','').replace('현대문학ㆍ2쪽ㆍ1만0원 내가 죽인 남자가 돌아왔다황세연 지음.','').replace('제공 양평군 양평 파이낸셜','')
        text = text.replace('오마이포토 박광수','').replace('대구인권사무소 제공','').replace('제공 함평군','').replace('대전S 자료','').replace('연합 정동철 통신원','').replace('제공 트랜스페어런트아츠','').replace('제공 양반스네이션','').replace('일간스포츠 안민구','').replace('제공 스튜디오드래곤','').replace('서울신문 박상현 코드미디어 디렉터','')    
        text = text.replace('불가능한 탈출 l','').replace('과 코너의 안쪽  ','').replace('제공 예술의전당','').replace('경기방송 최일 기자','').replace('M경기방송 문정진기자','').replace('경기방송 엄인용기자','').replace('e 은','').replace('경기방송 엄인용입니다','').replace('서울 서초경찰서 제공','').replace('강타 우주안 인스타그램','').replace('제공 의정부시','').replace('울산시민행동 제공','').replace('재판매 및 금지 울산','').replace('제공 안동','').replace('부산지방경찰청 제공','').replace('인천시교육청 제공','').replace('제공 유니버설뮤직','').replace('이번 연구 n t e e는 미국 노인병 학회 저널 에 실렸고 주간지 타임 이 보도했다','').replace('s이주형 경제산업 코리아','').replace('전자신문인터넷 박동선기자','').replace('국립부산과학관 제공','').replace('스포츠서울 신혜연기자','').replace('제공 부산시교육청','').replace('제공 안산시청','').replace('제공 한화갤러리아','').replace('제공 예술경영지원센터','').replace('여수해경 제공','').replace('자료 우주안 캡처 그룹','')
        text = text.replace('서울교통공사 제공','').replace('오마이 클레어함 기자','').replace('부산경찰청 제공','').replace('재판매 및 금지','').replace('장수 뉴시스 한훈 기자','').replace('장수군 제공','').replace('영상 제보는 보도에 반영되면 사례하겠습니다','').replace('는 여러분과 함께 만들어갑니다','').replace('많은 제보 부탁드립니다','').replace('자료제공 안산시','').replace('서울 박라경 에디터','').replace('해외 리얼돌 판매 사이트 캡처','')
        text = text.replace('야론 베르사노 좌 갤 가돗 우 갤 가돗 인스타그램','').replace('오정연 인스타그램','').replace('엔터테인먼트 공식 사이트 캡처','').replace('청주상당경찰서 제공','').replace('대전대 제공','').replace('스포츠서울 신재유기자','').replace('내용과 직접적인 관련이 없음','').replace('자료 제공 엘림넷','').replace('머니투데이 한지연기자','').replace('풀무원녹즙 온라인 프로모션 이미지 빅데이터','').replace('인터넷 화면 캡처','').replace('기업과 예술 ㅣ 예술경영','').replace('김태훈의 무릉서원 바로가기 곽아람의 독서알람 바로가기 김지수의 인터스텔라 바로가기 과식자들 바로가기 김광일의 입 바로가기','')
        text = text.replace('제공 여수시 ','').replace('이 연구결과는 미국 심장학회 저널 ','').replace('이 연구결과는 미국 신경학회','').replace('커버스토리 ㅣ','').replace('연제구청 제공','').replace('한국에너지공단 제공','').replace('제공 오륜정보산업학교','').replace('글 하이닥 상담의사 류기현 원장 한의사','').replace('차미혜 웹데일리 기자','').replace('제공 흥국생명','').replace('제공 제주항공','').replace('전문가 칼럼은 하이닥 의견과 다를 수 있습니다','')
        text = text.replace('제보는 전화 번이나 ','').replace(' 카카오톡 플러스 친구 제보','').replace('최신호 8월 1일 자 에 실렸다','').replace('검색하셔서 친구맺기를 하신 뒤 제보해주시기 바랍니다','')
        text = text.replace('w l국방일보 년 8월 1일3쪽','').replace('저작권자 한국군사문제연구원','').replace(' e 제9호 한국군사문제연구원 발행','').replace('제공 디에이치플레이엔터테인먼트','').replace('경기 화성병 오건호 기자','')
        text = text.replace('남구청 제공','').replace('연합 에너지경제신문 ','').replace('제공 비비엔터테인먼트','').replace('용인시 제공','').replace('이 연구 결과는 미국 심장병학회 저널 a g 최신 호에','').replace('발표됐으며','').replace('올바른 건강정보 알리기','').replace('등 외신이 전했다','').replace('바름정의경제연구소 제공','').replace('초록뱀미디어 제공','').replace('경남이주민센터 제공','').replace('광주시 제공','').replace('문성일 선임기자','').replace('금산군 제공','').replace('글 하이닥 의학기자 신예지 원장 한의사','').replace('한의사 침구과 전문의','').replace('보그 인스타그램','').replace('한전S 제공','').replace('연합 자료 청주 연합','').replace('s김덕권 사회문화 칼럼','').replace('태그호이어 광고 이미지 롯데쇼핑 제공','').replace('제이와이피엔터테인먼트 제공','').replace('원앤원 창업박람회 자료','').replace('일간스포츠 자료','').replace('건강보험심사평가원 제공','').replace('제공 광주시자원봉사센터','').replace('슈나이더일렉트릭 제공','').replace('아시아경제 김가연 인턴기자','').replace('오피니언 ㅣ 키워드이슈','').replace('이 은 경찰이 촬영한 영상의 캡처본','').replace('이 의 페이스북에도 실립니다','').replace('제공 브랜뉴에이','').replace('제공 젠지 이스포츠','')
        text = text.replace('설선정 인턴기자','').replace('김상훈현장을 넘어 구조까지 살펴보는 기자가 되겠습니다','').replace('제보를 주시면 달려갑니다','').replace('이메일 항상 주시하고 있습니다','').replace('삼성전자 제공','').replace('국립도서관 오마이 글 백창민 글 이혜숙 편에서 이어집니다','').replace('M경기방송 윤상식입니다','').replace('홍윤기 인턴기자','').replace('서효문 기자','').replace('제공 국립항창단','').replace('한겨레 자치소식','').replace('제공 부산시 ','').replace('강원도 고성에서 글 조아미','').replace('제공 논산시청','').replace('서윤 인스타그램','').replace('충북도 헤럴드경제 대전 이권형기자','')
        text = text.replace('제공 삼성전자 에너지경제신문','').replace('해당 이미지는 와 직접적인 관련은 없음 ','').replace('제주 유나이티드 제공 ','').replace('신정훈 의 전체 보기','').replace('아침마당 생방송 캡쳐','').replace('제공 대구시','').replace('아이즈 글 김리은 권나연 임현경','').replace('충북도 제공','').replace('재판매 및 금지','').replace('c 박혜진기자 ','').replace('제공 메이크어스 엔터테인먼트','').replace('제공 선미 인스타그램','').replace('은 와 직접적인 관련이 없습니다','').replace('자료제공 성남시','').replace('자료 제공 G마켓 빅데이터','').replace('국민리포트 윤근배입니다','')
        text = text.replace('시작페이지로 즐겨찾기 편집','').replace('서브가서브가크레소티 크레소티크레소티 새시안','').replace('제공 스톤뮤직엔터테인먼트','').replace('제공 오산새일센터','')
        text = text.replace('이미지투데이 주말리뷰','').replace('강민경 인스타그램 주말리뷰','').replace('조세금융신문 내 박스','').replace('중기이코노미 다른보기','')
        text = text.replace('전체 로그인 찾기','').replace('화면캡처 미우새','').replace('N 조창훈 김도형입니다.','').replace('맑은고딕 u i','').replace('바탕글 바탕','')
        text = text.replace('짠내투어 이시아 더 짠내투어','').replace('이 글은 이전 기고글과 연결됩니다.','').replace('스타 손민지 인턴기자','').replace('예능프로그램 마이 리틀 텔레비전 V방송화면 캡쳐','')
        text = text.replace('베이징에서 C 김희웅입니다.','').replace('도쿄에서 황보연입니다.','').replace('박선호 ㅣ','').replace('신문 권이민수 기자','').replace('극단 산수유 두산아트센터 제공','')
        text = text.replace('베이징에서 최영은입니다.','').replace('회사제공','').replace('이 는 한국농어민신문 에도 실립니다.','').replace('제공 카카오엠','').replace('당신이 잠든 동안 가 쏟아집니다.','').replace('그 중에서 반드시 챙겨야 할 매일 아침 브리핑 으로 전해드립니다.','')
        text = text.replace('연애의 참견 시즌제공','').replace('사람이 좋다 캡처','').replace('전국금융산업노동조합 홈페이지','').replace('엘런튜브 캡처','').replace('네이버 블로그 공유하기','').replace('카탈리스트 공식 홈페이지','').replace('제공 남양주경찰서','').replace('청와대 국민청원 게시글 갈무리','').replace('모바일한경 더보기','').replace('조권이 지난 8일 오후 열린 뮤지컬 제이미 원제 프레스콜에 참석해 공연 넘버 r n 하이라이트 장면을 시연했습니다.','')
        text = text.replace('아메리칸 항공 홈페이지 갈무리','').replace('스타 제공','').replace('제공 텐가','').replace('e카카오스토리','').replace('트위터 캡처','').replace('제공 다노','').replace('제공 휴먼다큐 사람이 좋다','').replace('청와대 국민청원 홈페이지 게시판 갈무리','').replace('대구시 제공','').replace('제공 삼성물산 패션 알라이아 빅데이터','')
        text = text.replace('제공 이디야커피','').replace('포스코기업시민 포스코 성과공유의 장 개최','').replace('총 비행시간은0시간이다','').replace('거창경찰서 아림지구대 배종현 순경 제공','').replace('제공 삼성물산 패션부문','').replace('제공 사람인','').replace('본 는 머니S 제1호 년 월9일 에 실린 입니다.','')
        text = text.replace('아래를 클릭하시면 고희정 주간베스트 동영상을 감상할 수 있습니다.','').replace('김유지 인스타그램','').replace('한전 럭비단 홈페이지','').replace('예고편 캡처','').replace('제공 진주경찰서','').replace('인구보건복지협회 제공','').replace('부산일보 홈 바로가기','').replace(' 이 는 언론사에서 생활 섹션으로 분류했습니다.','')
        text = text.replace('비즈플러스 이슈체크','').replace('창원대학교 전경 창원대 제공','').replace('제공 마운틴무브먼트','').replace('캠퍼스 잡앤조이 김혜선 인턴기자','').replace('제공 생활약속','').replace('김지호기자','').replace('한전 럭비단 홈페이지','').replace('스타 장은송 인턴기자','')
        
        text = text.replace('제공 경기연구원','').replace('제공 부산선관위 !','').replace('? 한국미혼모가족협회 한국미혼모가족협회 .','').replace('? 미혼모가족 미혼모가족 .','').replace('? 이지그립 이지그립 .','')
        text = text.replace('가 마음에 드셨나요? . 그럼에도 불구하고 베이비는 앞으로도 읽는데 불편한 광고는 싣지 않겠습니다.','').replace('낙태죄 반대 서명 0만 명 넘어 .','').replace('? 낙태죄 반대 서명 0만 명 넘어 ? 낙태죄 반대 서명 0만 명 넘어 ? 낙태죄 반대 서명 0만 명 넘어 네이버 블로그 .','').replace('? 낙태죄 반대 서명 0만 명 넘어 카카오스토리 .','').replace('확대 본문 텍스트 크게 축소 본문 텍스트 작게 ! .','')
        text = text.replace('v 중기 ㅣ 지금 중기에선 .','').replace('이창호 다른보기 .','').replace('? 맘스클래스 맘스클래스 .','').replace('? 광진파티올 광진파티올 .','').replace('? 고양이뿔 고양이뿔 ','')
        text = text.replace('? 대원제약 대원제약 .','').replace('? 육아교실 육아교실 .','').replace('? 광진맘스클래스 광진맘스클래스 .','').replace('? 초보운전 초보운전 .','').replace('? 산모교실 산모교실 .','')
        text = text.replace('? 육아교실 육아교실 .','').replace('? 손해보험 손해보험 .','').replace('? 프로미클래스 프로미클래스 .','').replace('','').replace('? 지음요가 지음요가 .','')
        text = text.replace('자료제공 문체부','').replace('? 부여 농업기술센터제공 .','').replace(' m e m g m m c m .','').replace('! 0명이 을 를 좋아합니다.','').replace('? 클릭아트코리아 ! .','')
        text = text.replace('? 새모델에 스테파니 리 생활건강 ! .','').replace('한국과학창의재단 제공','').replace('? 제공 생활건강 빅데이터 .','').replace('? 제공 아모레퍼시픽 빅데이터 .','').replace('홈페이지 캡쳐.','')
        text = text.replace('? 제공','').replace('! 페이지 본문내상단 .','').replace(' u i ! 인스타일 .','')
        text = text.replace('확대 본문 텍스트 크게 축소 본문 텍스트 작게 ! .','').replace('오마이 글 이성경 편집 김혜리','').replace('? 뉴시스 제공 빅데이터 .','').replace('! 페이지 애드인 키워드배너 .','').replace('! 에이플에이디 데일리시큐 중앙상단 .','')
        text = text.replace(' ? ?검색 ?c포털 .','').replace('? 실화탐사대 캡처 빅데이터 .','').replace('? 뉴시스 제공 빅데이터 .','').replace('? 홍주영기자 다른 보기 .','').replace('후플러스의 모든 콘텐츠 영상 는 저작권법의 보호를 받은바 무단 전재와 복사 배포 등을 금합니다.','')
        text = text.replace('? 이미지 엠넷 방송화면 일부캡처 ! .','').replace('? 캘리포니아 아몬드 협회 ! .','').replace('? 고등래퍼빅데이터 ','').replace('','').replace('? 이미지 엠넷방송 .','')
        text = text.replace('헌법재판소 낙태 헌법불합치 판결 E보기 김길수 사무총장 생명운동연합 네이버 블로그','').replace('헌법재판소 낙태 헌법불합치 판결 E보기 김길수 사무총장 생명운동연합 카카오스토리 .','').replace('헌법재판소 낙태 헌법불합치 판결 E보기 김길수 사무총장 생명운동연합 카카오스토리 .','').replace(' ?검색 ?c저녁 ','')
        text = text.replace('스포츠서울 남혜연기자 ','').replace('영상취재 민경철 영상편집 최아람','').replace('제공 쏘내추럴 생활건강','').replace('박차정 의사 학술심포지움 포스터 더불어민주당 경남도당 제공','').replace('제공 노랑풍선','').replace('이제 돌아섭니다 댓글에 공감','').replace('닐슨코리아 제공','').replace('보도에 오인환기자입니다','').replace('매일경제 스타투데이 유림 인턴기자','').replace('제공 G마켓','').replace('표지 b 룸 캡처 메디컬월드','').replace('인터넷 캡처','')
        text = text.replace('서울 뉴시스 전신 기자','').replace('개각 장관급 인사 내정자 프로필','').replace('원문 김현성의 페이스북','').replace('손태승닫기손태승 모아보기','').replace('오그도아 제공','').replace('서울 허경 기자','').replace('우창범 인스타그램','')
        text = text.replace('제공 창원중부경찰서','').replace('업체 홈페이지 화면 캡처','').replace('축산신문 기자','').replace('창원시 제공 ','').replace('빅히트엔터테인먼트 제공','').replace('최병일 여행 레저전문기자','').replace('인터뷰 이인선','').replace('남원 뉴시스 한훈 기자','').replace('자료 제공 현대경제연구원 아시아경제','').replace('서울신문 로이터 자료 연합','').replace('김휘린 연구사 제공','')
        text = text.replace('영상취재 오영춘 ','').replace('로이터통신 연합','').replace('영상편집 이승희  ','').replace('사공성근 기자의 보도입니다','').replace('한국경제 홍헌표입니다','').replace('일러스트 삽화 임종철 디자인기자','').replace(' 제공 바이서우','').replace('제공 완도군 완도 스포츠서울','').replace('제공 법무부','').replace('제공 저스트뮤직','').replace('광주 전남지방중소벤처기업청 제공','').replace('제공 미나트','').replace('국사편찬위원회 제공','').replace('제공 바나나몰','').replace('제공 보미라이','').replace('전주시 덕진구청 제공','').replace('부산관광공사 제공','').replace('제공 안성시','').replace('목포시 의회 제공 ','').replace('김정순 작가 제공','').replace(' 안성시 제공','').replace('국립광주박물관 제공 뉴시스','').replace('킹엔터테인먼트 제공','').replace('문화재청 제공','').replace('메이크어스엔터테인먼트 제공','').replace('부산시 제공 ','').replace('제공 카페그','').replace('간송미술관 제공','').replace('제공 아시아문화전당','')
        text = text.replace(' 부산관광공사 제공','').replace('한국청소년상담복지개발원 제공','').replace('공주대 제공','').replace('서귀포시 제공','').replace('하이트진로음료 빅데이터','').replace('경남S 자료','').replace('국민리포트 회 클립영상','').replace('국민리포트 민지선입니다','').replace('취재 민지선 촬영 이승주','').replace(' 기자 킴 바커','').replace('','')
        text = text.replace('일산 강재훈 선임기자 ','').replace('광주 김봉규 선임기자','').replace('글로벌 청원사이트 체인지닷오알지','').replace('일 체인지닷오알지에 따르면 백윤진 u','').replace('영상취재 유지영','').replace('배미래 인턴기자','').replace('영상취재 한재훈 영상편집 문철학 저작권자 ','').replace('특별기획 복원된 백범 김구 선생의 목소리로 내가 쓴 글 들어보기! 엠빅 카를루스에게는 O슛이 껌이었다','').replace('영상취재 강미이 전주 저작권자','').replace('영상취재 임재석 충북 저작권자 ','').replace('영상취재 정연철 영상편집 이정섭','').replace('제공ㅣ초록뱀미디어 스포티비','').replace('제공 파란미디어드라마 ','').replace('ㅣ 입력일 ','').replace('로그인을 체크하는 로직','').replace('농협 제공 ','').replace('캡처 울산저널','').replace('회원가입이 실패하였습니다','').replace('영상편집 원형희','').replace('농협 제공 ','').replace('통일부 청사 내부 핌','').replace('개인정보 수집 및 이용 동의 레터 발송을 위해 최소한의 개인정보를 수집 이용합니다','').replace('수집한 정보는 메일 발송 외의 목적으로 이용하지 않으며 레터 서비스가 끝나거나 이용자가 구독을 해지하면 즉시 파기합니다','')
        text = text.replace('제공 삼성전자','').replace('오마이 윤현 기자','').replace('제주동부경찰서 제공','').replace('국방기술품질원 제공','').replace('구미시 제공','').replace('크리스 와일드 제공 ','').replace('인터뷰의 저작권은 ','').replace('인용보도 시 를 밝혀주시기 바랍니다','').replace('산청엔청복지관 라온교실 문화체험 실시입력','').replace('제공 엠넷','').replace('제공 와이드앵글','').replace('로스앤젤레스 연합 옥철 특파원','').replace('제공 앙코르','').replace('영상취재 김정섭 심동영 영상편집 김종석 ','').replace('제공 민주노총울산본부','').replace('머니투데이 변휘 기자 제공 국민은행','').replace(' 님의 공유 게시물님7월오전','').replace('제공 화성시','').replace('제공 장흥군','').replace('고 김계석 선생 유족 제공','').replace('제공 스티유','').replace('제주도청 제공','').replace('제공 안산시','').replace('제공 국민은행 ','').replace('제공 데이즈','').replace('C今日中国人 怖 近 撮 誰 呼 撮影 中止 入 人','').replace('p 愛宕2夏 3日目 s','').replace('화성 김봉규 선임기자','').replace('삼성물산 패션부문 빅데이터','')
        text = text.replace('B 화면 캡처','').replace('전주시설공단 제공','').replace('제공 강동경희대병원','').replace('자료 제공 빅데이터','').replace('이미지는보야지 투 자라섬 포스터','').replace('권혁재 전문기자','').replace('신선아 디자이너 대전 지역 잡지 보슈','').replace('제공 K타이거즈엔터테인먼트','').replace('재판매 및 금지','')
        text = text.replace('놀라운 토요일 방송캡쳐','').replace('f c e a u t','').replace('전성우 방송화면','').replace('한 달에 1번 영상 콘텐츠로 찾아갑니다','').replace('은 와 직접 관련 없음','').replace('박준 작가 제공','').replace('제공 조선','').replace('제공 남양주시','').replace('한반도 사회문화 칼럼','').replace('제공ㅣ앳스타일 스포티비 ','').replace('디지털데일리 홍하나기자','').replace(' 충남 당진시청 자료','').replace('인사혁신처 제공','').replace(' 제공 코오롱인더스트리C부문 슈콤마보니','').replace('서울 이철 기자','').replace('국토교통부 제공','').replace(' 경기도 제공','').replace('제공 롯데하이마트 빅데이터','').replace('제공 싱글즈','').replace('제공 금호타이어','').replace('하이마트 제공','').replace('제공 부산지방변호사회','').replace('싱글즈 제공','').replace('사상경찰서 제공','').replace('제공 전주','').replace('영화사 리듬앤블루스 제공','').replace('이병준 사회2팀 기자','')
        text = text.replace('호날두 S캡처','').replace('일간스포츠 이소영','').replace('제공 나인','').replace('제공 여성기업종합지원센터 ','').replace('경향신문 자료','').replace('대전지방경찰청 제공','').replace('제주지방경찰청 제공','').replace('제공 국회기자단 가칭','').replace('한옥희 감독 국립 아시아문화전당 제공','').replace('이번 연구는 영국 암 저널 ','').replace('신청 한국장학재단 누리집','').replace('및 한국장학재단 휴대용 앱 더 자세히 알아보기','').replace(' 일단 1초동안 까맣게 됐다가불투명도로 간다','').replace('로그인 후에 참여하실 수 있습니다','').replace('보내는 형식을 지정한다','').replace('오승현기자 서울경제','').replace('연합 제공','').replace('제공 모스트콘텐츠K드라마 콘서트 ','').replace('연합 자료 서울경제 ','').replace('자료 제공 증권','').replace('지정한 페이지애서 리턴한 값 또는 값 을 받아온다','').replace('에러인 경우 에러메시지 알럿','')
        text = text.replace('제공디즈니','').replace('스타 이시연 인턴기자','').replace('패션엔 류숙희 이민지','').replace('제공 방구석1열','').replace('더팩트ㅣ김세정 기자 ','').replace('안재현 인스타그램 헤럴드경제','').replace('작곡가 오상훈 ','').replace('울산중부경찰서 전경 뉴시스 울산 뉴시스','').replace('관련 소식은 홈페이지','').replace('수원시 제공','').replace('제공 한국도자재단','').replace('제공 전남교육청','').replace('광주신세계백화점 제공','').replace('신세계백화점 제공','').replace('롯데백화점 전주점 제공','').replace('에너지경제신문 에너지경제 지면 다음은 8월 일 목요일 에너지경제신문 지면이다','').replace('관련를 더 보시려면','').replace('제공 생활건강 ','').replace('제공ㅣN 신혼일기','').replace('이베이코리아 G제공','').replace('인용된 역사서에 대한 코멘트1','').replace('C에브리원 비디오스타 의 이유정','').replace('제공 C플러스미디어','').replace('제공 ㈜오엠인터랙티브','').replace('글항아리 제공','').replace('매일경제 스타투데이 이누리 객원기자','').replace('안동시 제공','')
        text = text.replace('안정혜 지음 아이브이피 1만원','').replace('어촌어항공단 제공','').replace('전민식 지음 마시멜로 2쪽 1만 원','').replace('주석주석김승곤 조선의열단의 창립과 투쟁 전사 5호 3쪽','').replace('김학철 최후의 분대장9쪽 문학과 지성사 김원봉 건립동방각민족우의적신기초 조선의용대통신 기쪽','').replace('김삼웅의 인물열전 운암 김성숙 평전은 매일 여러분을 찾아갑니다','').replace('이 논문은 대한골다공증학회 저널 i i 최신호에 게재됐다','').replace('포천소방서 제공','').replace('윤성호기자 국민일보','').replace(' 이달의보도상 ','').replace('이달의보도상 최우수상 ','').replace('한국기자협회 무단전재 재배포금지','').replace('노민규기자 중부일보','').replace('장진영기자 중앙일보','').replace('이성철기자 인천일보','').replace('이진우기자 기호일보 ','').replace('한국기자협회','').replace(' 무단전재 재배포금지 이달의보도상 우수상 ','')
        text = text.replace('한국경제 라이온봇 기자','').replace('제공 메이크어스엔터테인먼트','').replace('그래프 내츄럴엔도텍 일봉 차트 표 거래원 상위기준 내츄럴엔도텍라이온봇 기자 한국경제 본 는 한국경제와 금융 전문기업 씽크풀 이 실시간으로 작성한 입니다','').replace('제공 블랙야크 마운티아','').replace('m갤럭시노트최소ㆍ최대 지원금','').replace('포천소방서 제공','').replace('손형주기자 ','').replace('이진우기자 기호일보  ','').replace('이동훈기자 머니투데이','').replace('제공 신세계인터내셔날','').replace('출판사 제공','').replace('스타로드엔터테인먼트 제공','').replace('엔터테인먼트 제공','').replace('김지훈기자 국민일보','').replace('제공 나누다키친공유점포플랫폼','').replace('머니투데이 중기협력팀 김태윤','').replace('뉴델리 연합 김영현 특파원 ','').replace('메타커뮤니케이션즈 제공','').replace('폴리 이경민 수습기자 ','').replace('충주 뉴시스 전신 기자','').replace('노틸러스 애뉴얼 캘린더 문페이즈 f','').replace('파텍필립 제공','').replace('엑스포츠 박소연 인턴기자','').replace('인터뷰 및 를 인용보도할 때는 가톨릭평화방송 을 정확히 밝혀주시기 바랍니다','')
        text = text.replace(' 예능프로그램 서울메이트방송화면 캡쳐','').replace('경향신문 기자칼럼인 기자메모','').replace('링크링크링크링크링크링크링크링크링크링크','').replace('글 하이닥 의학기자 김기호 원장 성형외과 전문의','').replace('장흥군 제공','').replace('닥스 액세서리 제공','').replace('제공 주 스타벅스커피 코리아 빅데이터','').replace('장순영 빅데이터 기자','')
        text = text.replace('제공 집사부일체','').replace('제공 서울경제','').replace('제공ㅣ엔터테인먼트','').replace('네이버 제공','').replace('제공G엔터테인먼트','').replace('김정근 선임기자 ','').replace('국회사무처 제공','').replace('부산항만공사 제공','').replace('제공 대구남부경찰서','').replace('나무엑터스 제공','').replace('제공 코스모폴리탄','').replace('행정안전부 제공','').replace('롯데백화점 빅데이터','').replace('제공 광주광역시','').replace('일 현지시각 엔가젯은','').replace('킴 카다시안 로이터 핌 ','').replace('자료 삼성생명','').replace('알림 시각장애인 웹접근성 보장을 위한 대체텍스트입니다','').replace('제공 피네이션','').replace('보령시 제공','').replace('제공 이노텍','').replace('방송일시 년 8월 일 월','').replace('진 행 소영선 프로듀서','').replace('출 연 유순덕 경기도청소년상담복지센터장','').replace('소영선 프로듀서 이하 소','').replace('ㅣ안재현 인스타그램','').replace('도쿄에서 C 박글 하이닥 의학기자 임은교 원장 한의사진주입니다','').replace('유튜브 화면캡처','').replace(' 그래픽 한국일보','')
        text = text.replace('박연직 사회2부 선임기자','').replace('박민정 인스타그램','').replace('머리카락 한 올 ','').replace('한겨레 자료','').replace('제공엔터테인먼트','').replace('머니투데이 남수현 인턴','').replace('제공 엔터식스패션쇼핑몰','').replace('제공 에몬스가구기업','').replace('제공 엣나인필름','').replace('해당 이미지는','').replace(' 내용과 무관한 자료입니다','').replace('한겨레노 땡큐! ','').replace('수자원공사 제공','').replace('전주교구 홍보국 제공','').replace('제공대구시','').replace('부천시제공','').replace('은 내용과 무관 뉴시스','').replace('인천시 제공','').replace('자이르 보우소나루 브라질 대통령 로이터 핌','').replace('영상취재 최정현 ','').replace('최소라 과학팀 기자','').replace('춘천경찰서 제공','').replace('제공 시타','').replace('자료 뉴시스','').replace('전남소방본부 제공','').replace('제공 온스테이지 디깅클럽','').replace('김기철 조선일보 ','').replace('글 하이닥 의학기자 임은교 원장 한의사','').replace('알렉스 카츠 e t','').replace('코리아나미술관 제공','').replace('제공 연천군','').replace('풀무원한가위 선물세트 이미지','').replace('제공 뉴런뮤직 ','').replace('신문들의 헤드라인 다','').replace('아쿼틴트 x㎝','')
        text = text.replace('락앤락 플레이스엘엘 롯데몰 수지점 빅데이터','').replace('본 는 날씨 전문 매체 온케이웨더','').replace('에서 제공하고 있습니다','').replace('니프코리아 빅데이터','').replace('제공 무브먼트제네레이션','').replace('스포티비 이제 시작입니다','').replace('제공 주 이연생활뷰티','').replace('','')
        text = text.replace('에스메 반 룬네덜란드 i ','').replace('영상취재 박현철 영상편집 김종우 앵커 송욱 특파원 ','').replace('黑熊和姊姊的飛飛 ','').replace('대구소방본부 제공','').replace('구혜선 인스타그램','').replace('베이비 좋은 후원하기','').replace('이선민 조선일보 선임기자','').replace('암보험과건강하게사는이야기','').replace(' 스포츠조선','').replace('충남도 제공','').replace('조달청 제공','').replace('제공 벤처기업협회','').replace('제공l무브먼트제너레이션 스포티비 ','').replace('제공 X','').replace('런던 서울 연합 박대한 특파원 ','').replace('세 딸의 엄마가 살 수 있는 세상은 오마이 ','').replace('안재원 대표 페이스북 바로 가기','').replace('제공 경남제약 ','').replace('제공 문화체육관광부','').replace('조르쥬레쉬 빅데이터','').replace('i t e a','')
        
        
        
        
        
        try: 
            text = text[:text.rindex('다.')+2]
            text = text.split(sep='.')
            for _ in text:
                if len(re.sub(r'\s{2,}', ' ', _)) > 11:
                    ttlist.append(re.sub(r'\s{2,}', ' ', _))
                    
            text = ".".join(ttlist) + "."
            return text
        except:
            return 0

In [171]:

df = pd.read_excel(xlist[243])
print(len(df))

549


In [170]:

"""
유미 0-91 지은 92-182 은서 232-269 유리 274-364
"""

df = pd.read_excel(xlist[242]) # 여기에 숫자 바꿔가면서 하면 돼
for i in range(0,409): # 한 번에 다 출력해서 하던가 아니면 하나하나 출력해서 하던가 해
    text = df.iloc[i, 3]
    print(replace_sub(text))

생활개선회의령군연합회 현장교육 .
의령군은 일 미래농업을 이끌어갈 여성지도자 생활개선회원들을 대상으로 올바른 역할 수행과 역량 강화를 위한 현장교육을 했다.
이번 현장교육은 여수시 돌산갓대율마을 갓김치가공사업장 견학하는 등 다양한 소득 창출의 방안을 벤치마킹했다.
생활개선의령군연합회 현장교육입력 .
생활개선회의령군연합회 현장교육 .
의령군은 일 미래농업을 이끌어갈 여성지도자 생활개선회원들을 대상으로 올바른 역할 수행과 역량 강화를 위한 현장교육을 했다.
이번 현장교육은 여수시 돌산갓대율마을 갓김치가공사업장 견학하는 등 다양한 소득 창출의 방안을 벤치마킹했다.
생활개선회의령군연합회 현장교육 .
의령군은 일 미래농업을 이끌어갈 여성지도자 생활개선회원들을 대상으로 올바른 역할 수행과 역량 강화를 위한 현장교육을 했다.
이번 현장교육은 여수시 돌산갓대율마을 갓김치가공사업장 견학하는 등 다양한 소득 창출의 방안을 벤치마킹했다.
동치미 손헌수 박수홍 속풀이쇼 동치미 캡처 동치미 손헌수가 박수홍 때문에 여자친구와 헤어졌다고 밝혔다.
일 오후 방송된 예능프로그램 속풀이쇼 동치미 이하 동치미 에서는 개그맨 손헌수가 박수홍 때문에 결혼을 못하는 이유를 공개했다.
이날 손헌수는 제가 어렸을 때 오래 만난 여성이 있었다.
결혼까지 생각했다 라며 굉장히 예뻤는데 술 담배를 즐기고 거짓말을 좋아했다 라고 입을 열었다.
이어 저랑 만나며 다른 남성을 만나고 다이어리에 쓰더라 라며 사랑이 너무 평탄해도 재미가 없다.
버라이어티 하고 정이 붙어서 결혼까지 생각했는데 그 정을 박수홍 선배가 떼줬다 라고 말했다.
그는 제가 유일하게 말을 듣는 사람이 박수홍 선배였다 라며 선배님이 화장실을 갔다 오시더니 저 친구 계속 만날거냐며 헤어졌으면 좋겠다고 하더라 라고 당시 상황을 설명했다.
손헌수는 맞담배를 요구했다더라.
저 여자는 아니라고 해서 여자친구와 헤어졌다 라고 덧붙였다.
 주말드라마 타인은 지옥이다 서울 임시완이 지옥의 고시원에 입성했다.
일 오후 처음 방송된 주말드라마 타인은 지옥이다 극본

우리가 흔히 말하는 오덕 은 해당 분야를 잘 아는 마니아 를 뜻함과 동시에 팬덤 등 열정을 상징하는 말로도 통합니다.
조선은 애니메이션 만화 영화 게임 등 오덕 문화로 상징되는 팝컬처 콘텐츠에 대한 이야기를 풀어가고자 합니다.
어린시절 열광했던 인기 콘텐츠부터 최신 팝컬처 분야 핫이슈까지 폭넓게 다루머 오덕의 가려운 부분을 긁어 줄 예정입니다. 신세기X 사이버포뮬러 는 년 애니메이션을 통해 소개된 가상의 자동차 레이싱 경기를 소재로 한 작품이다.
주인공 하야토와 사이버포뮬러 머신 아스라다X.
사이버포뮬러는 근미래를 무대로 주인공과 대화가 가능한 인공지능 이 탑재된 자동차로 경기를 펼치는 것이 특징이다.
등장 레이싱 차량은 현실 세계에서 가장 빠르다는 포뮬러원 F머신의 시속 ㎞를 훌쩍 뛰어넘는다.
주인공 카자미 하야토가 탑승했던 아스라다 는시속 ㎞로 내 달린다.
실제 레이싱에서는 불가능한 공중 드리프트 리프팅 턴 과 분신 공격인 미라쥬 턴 이 사이버포뮬러 팬들의 손에 땀을 쥐게 했다.
말도 안되는 이 기술은 아직도 많은 애니메이션 팬들의 입에 오르내리고 있다.
리프팅턴과 미라쥬턴 연출 화면.
사이버포뮬러 애니메이션은 주인공 하야토가 아버지가 만든 레이싱카 아스라다로 세기 F이라 평가받는 사이버포뮬러 월드 그랑프리에 참가해 우승을 이어간다는 내용을 담았다.
주인공은 년부터 년까지 9년간 이어진 애니메이션 시리즈를 통해 소년에서 성인으로 성장해 가는 것이 특징이다.2년 주기로 열리는 대회에 맞게 캐릭터도 나이를 먹어 가는 셈이다.
사이버포뮬러에 사용되는 레이싱카는 수소엔진 초전도 모터 등 친환경 엔진을 사용한다.
또 레이싱을 승리로 이끌기 위해 가 탑재된다.
무대가 되는 사이버포뮬러 대회는 시리즈는 애니메이션 속 세상을 무대로 년 오리지널비디오애니메이션 더블원부터 사가 는 년부터 년 경기를 무대로 삼고 있다. 시리즈는 주인공 하야토가 사이버포뮬러 그랑프리에 참가해 성장해 가는 과정을 신 은 하야토가 아닌 브리드 카가 를 주인공으로 내세워 레이싱 서킷의 젊은 제왕 으